# Description

## Business Problem

This project was created to solve a real business problem. The company had a table of customers whose addresses were in string field where the users could write the whole addresses themselves. After some changes in business needs, the company had to split this field into proper columns: suburb, postcode, state, street name, street number, etc.

Since the company had more than 10k customers in the table, the script was created to help with this task. Also, the system used by the company would continue to be the same, that is, just one big string field for the address, so the script would still have to be used to continue to split the addresses into the columns.

To be able to complete the task, the fields had to be split, there was a separate address table to store the addresses and generate an ID that would also be used. To store the addresses in this table, I had to use other dimension tables to get the ID of the suburb and postcode. 

The expected address format in the string was:

```
                {Address1}
                {Address2}
{StreetNumber} {StreetName} {StreetType}
       {Suburb} {State} {Postcode}
```

One example of this would be:

```
{P/L & GS Engineering & Constructions Australia P/L & Webuild SPA}        ---> Address1
                    {T/As Spark NEL DC JV}                                ---> Address2
            {Level 1, 436-438} {Johnston} {Street}                        ---> StreetNo, Name and Type
                    {Abbotsford} {VIC} {3067}                             ---> Suburb, State, Postcode
```
We also had the PO Boxes and Privete Bags addresses:

```
  PO Box 1182
Rosebery NSW 1445
```

The only difference is that the whole first line of PO Boxes was intended to be in a specific column.

If the address was not in this format, the script would not work for it, and it should be updated.

The sript was able to update more than 90% of the cases.

## Problems found while developing the project

### Problems that could be fixed creating functions

As the field was a big whole field, where the users could write the addresses as a whole, it was very easy to make mistakes and typos.

Here are some examples of challenging existing cases:

- Name of the suburb right but with the wrong postcode:

```
   PO Box 6499
Shepparton VIC 3631  ---> correct postcode would be 3630
```

- Name of the suburb wrong but with the correct postcode:

```
   PO Box 6499
Sheppaton VIC 3630  ---> correct suburb would be Shepparton
```

This problem happened because to split the fields, I needed to find the ID of the suburb and postcode in a table, so if they did not match, I had a problem.

- Order of the suburb name is different:

```
        PO Box W223
WESTFIELD PARRAMATTA NSW 2150   ---> it should be PARRAMATTA WESTFIELD
```

- Number of words of the suburb did not match the number of words the system had for that suburb and postcode:

```
    28 Peachtree Road
WESTFIELD PENRITH NSW 2750     ---> it should be PENRITH
```

To fix these problems, 3 functions were created:

1. find_matching_suburb:

If there was no exact match with postcode and suburb in the system's list, the code uses this function to try to find a suburb with a similar name. It uses the fuzzywuzzy library, which allows me to find a percentage of match between 2 words. I set that the words had to have at least 80% of a match for the suburb to be chosen.

For this one, the number of words in the suburb had to be the same. For example, WESTFIELD PENRITH would not match with PENRITH.

2. find_matching_suburb_split

If the first function did not work, the code tries to find the same percentage of match, but now taking into account that the suburb could have more or less words than what was written by the user. So it tries to find the best match for postcode and suburb.

3. find_matching_suburb_split_state

The last step if the first 2 ones do not work is to also use the state to try to find the best match. Now the code uses both suburb and state to try to find the best match.

If none of these work, the code generates an error and adds the customer to the skip list, and moves on to the next one.

### Problems that caused the script to skip the address

Here are some examples of problems that the code could not fix, so when they happened, the code generates an error, add the error and the customer id to the error dataset.

- No postcodes

```
Level 18 Darling Park
2/201 Sussex Street
 Sydney NSW
```

- Structure of the address incorrect:
```
Level 20
1 Market Street
Sydney NSW 2000 Australia
```

- No state found

```
17 Wilga Road
Caringbah South 2229
```

- No street number

```
Robertson Street
Kurrajong NSW 2754
```

- No street name

```
Bldg 505, Room 281
Darlington  NSW  2006
```

These are just some cases, we had much more, but it's enough to ilustrate the problems.

# Importing the libraries

In [1]:
import pandas as pd
import pyodbc
import re
from fuzzywuzzy import fuzz
from datetime import datetime
import os
import time
import xlsxwriter
import requests

# Creating the functions

In [2]:
# Defining the function the will be used to find the the index of the last number of the string

# The street name starts after the last number

def find_last_number_index(input_string):
    # Find all numbers and single letters right after numbers in the string
    matches = re.finditer(r'(\d+[A-Za-z]?)(?![A-Za-z])', input_string)
    
    # If there are no matches, return -1
    if not matches:
        return -1

    # Find the last match
    last_match = None
    for match in matches:
        last_match = match
    
    if last_match:
        last_value = last_match.group()
        last_value_index = input_string.rfind(last_value)
        return last_value_index + len(last_value) - 1
    else:
        return -1

In [3]:
# Function to find a suburb match based on fuzzy matching
def find_matching_suburb(suburb, postcode, suburbs):
    best_match = None
    best_match_score = 0
    
    # Tokenize and sort the words in the input suburb
    input_suburb_words = sorted(suburb.split())

    for index, row in suburbs.iterrows():
        current_suburb = row['SuburbName']
        current_postcode = row['Postcode']
        
        # Tokenize and sort the words in the current suburb
        current_suburb_words = sorted(current_suburb.upper().split())

        suburb_match_score = fuzz.partial_ratio(input_suburb_words, current_suburb_words)
        
        # You can adjust this threshold as needed to consider a match
        if suburb_match_score > best_match_score and suburb_match_score >= 80 and postcode == current_postcode and len(current_suburb.split()) == len(suburb.split()):
            best_match = current_suburb
            best_match_score = suburb_match_score

    return best_match

In [4]:
# Function to find a suburb match based on fuzzy matching
def find_matching_suburb_split(suburb, postcode, suburbs):
    best_match = None
    best_match_score = 0

    for index, row in suburbs.iterrows():
        current_suburb = row['SuburbName']
        current_postcode = row['Postcode']

        suburb_match_score = fuzz.partial_ratio(suburb, current_suburb.upper())
        
        # You can adjust this threshold as needed to consider a match
        if suburb_match_score > best_match_score and suburb_match_score >= 80 and postcode == current_postcode:
            best_match = current_suburb
            best_match_score = suburb_match_score

    return best_match

In [5]:
# Function to find a suburb match based on fuzzy matching
def find_matching_suburb_split_state(suburb, state, postcode, suburbs):
    best_match = None
    best_post_code = None
    best_match_score = 0
    best_postcode_diff = float('inf')  # Initialize with positive infinity

    for index, row in suburbs.iterrows():
        current_suburb = row['SuburbName']
        current_state = row['StateName']
        current_postcode = row['Postcode']

        # Calculate the similarity score between the input suburb and the current suburb
        suburb_match_score = fuzz.partial_ratio(suburb, current_suburb.upper())
                
        # You can adjust this threshold as needed to consider a match
        if (
            suburb_match_score >= best_match_score
            and suburb_match_score >= 80
            and current_state == state
        ):
            # Calculate the absolute difference between the current postcode and the target postcode
            postcode_diff = abs(int(current_postcode) - int(postcode))
            
            # Check if the current suburb has a postcode closer to the target postcode
            if postcode_diff < best_postcode_diff:
                best_match = current_suburb
                best_match_score = suburb_match_score
                best_postcode_diff = postcode_diff
                best_post_code = current_postcode

    return best_match, best_post_code

# Creating the objects that will be used in the code

## Connection settings

In [6]:
server = 'sampleserver'
database = 'sample_database'
username = 'sampleUser'
password = 'SamplePassword'

## Columns for the datasets

In [7]:
columns = ['Customer_Code', 'ID', 'Billing_ADR_Address_ID', 'Billing_Address', 'Billing_StreetNo', 'Billing_StreetName', 'Billing_StreetType',
           'Billing_Suburb', 'Billing_State', 'Billing_Postcode', 'Billing_Customisable_Value', 'Billing_Address2', 
           
           'Delivery_ADR_Address_ID', 'Delivery_Address', 'Delivery_StreetNo','Delivery_StreetName', 'Delivery_StreetType', 'Delivery_Suburb',
           'Delivery_State', 'Delivery_Postcode','Delivery_Customisable_Value' ,'Delivery_Address2']

columns

['Customer_Code',
 'ID',
 'Billing_ADR_Address_ID',
 'Billing_Address',
 'Billing_StreetNo',
 'Billing_StreetName',
 'Billing_StreetType',
 'Billing_Suburb',
 'Billing_State',
 'Billing_Postcode',
 'Billing_Customisable_Value',
 'Billing_Address2',
 'Delivery_ADR_Address_ID',
 'Delivery_Address',
 'Delivery_StreetNo',
 'Delivery_StreetName',
 'Delivery_StreetType',
 'Delivery_Suburb',
 'Delivery_State',
 'Delivery_Postcode',
 'Delivery_Customisable_Value',
 'Delivery_Address2']

## Error Report Dataset

In [37]:
billing_error_columns = ['Customer_Code', 'ID', 'Billing_Address', 'Billing Error Type']

delivey_error_columns = ['Customer_Code', 'ID', 'Delivery_Address', 'Delivery Error Type']

Billing_Error_Report = pd.DataFrame(columns=billing_error_columns)

Delivery_Error_Report = pd.DataFrame(columns=delivey_error_columns)

Billing_Error_Report

,Customer_Code,ID,Billing_Address,Billing Error Type


## Start time and Last Modified Date in the TBL_DBT_Address_Updates_Log table

In [9]:
start_date = datetime.now()

To assist with the code, a table was created that would insert the information about the script runs. Every time the script runs, at the end, it inserts into a table information such as start datetime, end datetime, last modified datetime, number of rows affected, etc.

This part would make a connection with SQL Server to get the information about the last time the customer table was modified when the script ran for the last time. So it would make a query to the customer table with the modifications after that time.

I changed the connection to a file connection for studies proposes.

In [19]:
# Establish the connection with Windows authentication

#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database
#cursor = conn.cursor()

select_last_date = """
SELECT MAX([Last_Modified_Date]) AS [Last_Modified_Date]
  FROM [sample_database].[dbo].[TBL_DBT_Address_Updates_Log]
"""

select_last_date = pd.read_excel("TBL_DBT_Address_Updates_Log.xlsx")

# Convert 'Last_Modified_Date' column to datetime format
select_last_date['Last_Modified_Date'] = pd.to_datetime(select_last_date['Last_Modified_Date'])

# Get the maximum date
select_last_date = select_last_date['Last_Modified_Date'].max()

#cursor.execute(select_last_date)
#last_finish_date = cursor.fetchone()[0]

#cursor.close()
#conn.close()

last_finish_date = select_last_date

last_finish_date = last_finish_date.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

last_finish_date

'2025-01-22 16:30:32.123'

## Last Modified Date

This part basically gets the last modified date from the audit table, which will be used to make some comparisons at the end.

In [17]:
# Establish the connection with Windows authentication
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
)

# Create a cursor object to interact with the database
cursor = conn.cursor()


select_modified_dates = f"""
SELECT MAX([Audit_DateTime]) AS Max_Audit_DateTime
FROM [sample_database].[dbo].[VA_DBT_Master]
WHERE [Audited_Field] IN ('Billing_Address', 'Delivery_Address') 
AND Operation IN ('U', 'I') 
AND Audit_DateTime > '{last_finish_date}'
AND Audit_User NOT LIKE 'TableauAdminAccess'

"""

cursor.execute(select_modified_dates)
last_modified_date = cursor.fetchone()[0]

cursor.close()
conn.close()

if last_modified_date is None:
    last_modified_date = last_finish_date
else:
    last_modified_date = last_modified_date.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

last_modified_date

'2025-01-24 15:13:29.240'

# Separating the addresses

## Billing Address

### First Group: the standard data

This part would make a connection with the database to get the Billing Address of the customers. Some parts were replaced for studies purposes.

The original data was modified to preserve sensitive information.

In [33]:
# Establish the connection with Windows authentication
#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database
#cursor = conn.cursor()


select_billing_first = f"""

WITH UpdateSelection AS (

SELECT DISTINCT [Source_Id],[Audit_DateTime]
FROM [sample_database].[dbo].[VA_DBT_Master]
WHERE [Audited_Field] IN ('Billing_Address') 
AND Operation IN ('U', 'I') 
AND Audit_DateTime > '{last_finish_date}'
AND Audit_User NOT LIKE 'TableauAdminAccess'

AND [After_Value] IS NOT NULL AND [After_Value] NOT LIKE 'TBA%' AND NOT

([After_Value]  LIKE '%Locked Bag%' OR [After_Value] LIKE '%PO BOX%' OR [After_Value] LIKE '%P.O. BOX%' OR [After_Value] LIKE '%P.O BOX%'
 OR [After_Value] LIKE '%P O Box%' OR [After_Value] LIKE '%Private Bag%')

)

SELECT     
    [Customer_Code],
    [ID],
    REPLACE([Billing_Address],CHAR(13)+CHAR(10),'_') AS [Billing_Address]
FROM [sample_database].[dbo].[DBT_Master]

WHERE 

[Billing_Address] IS NOT NULL AND [Billing_Address] NOT LIKE 'TBA%' AND NOT

([Billing_Address]  LIKE '%Locked Bag%' OR [Billing_Address] LIKE '%PO BOX%' OR [Billing_Address] LIKE '%P.O. BOX%' OR [Billing_Address] LIKE '%P.O BOX%'
 OR [Billing_Address] LIKE '%P O Box%' OR [Billing_Address] LIKE '%Private Bag%')

AND [ID] IN (SELECT DISTINCT [Source_Id]
            FROM UpdateSelection)
            
        """


#df = pd.read_sql_query(select_billing_first, conn)

#cursor.close()
#conn.close()

df = pd.read_excel("Billing Address.xlsx")

df

,Customer_Code,ID,Billing_Address
0,9NBRZAGLJDHKHFBUILDING,11551,"Suite 4, Level 1, 374 Church Street _Parramatt..."
1,9UNFBFE,11505,"Suite 802, Level 8, 24 Montgomery Street _Koga..."
2,9JZYZDTADNEKEACJ,11493,Saltspray Close_Redhead NSW 2290
3,9LVRNBVZENGPROJECTS,11480,"50 Fitzroy Street_Carrington, 2294"
4,9ENWOUWMENGINEERING,11479,Level 18 Darling Park_2/201 Sussex Street_ Syd...
...,...,...,...
136,CLQRGPUI,370,"48 Toh Guan Road East_#04-118, Enterprise Hub_..."
137,RUXHWWENGINEERING,10645,National Parks and Wildlife Service_Locked Bag...
138,GYUFABBUILDING,4633,(T/As Lendlease Bouygues Joint Venture) _T/As ...
139,LTITRKTRZFQSERVICES,1175,Locked Bag 18_Port Melbourne VIC 3207


In [34]:
billing_first_group = pd.DataFrame(columns=columns)

billing_first_group['Customer_Code'] = df['Customer_Code']
billing_first_group['ID'] = df['ID']
billing_first_group['Billing_Address'] = df['Billing_Address']

billing_first_group

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2
0,9NBRZAGLJDHKHFBUILDING,11551,NaN,"Suite 4, Level 1, 374 Church Street _Parramatt...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9UNFBFE,11505,NaN,"Suite 802, Level 8, 24 Montgomery Street _Koga...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9JZYZDTADNEKEACJ,11493,NaN,Saltspray Close_Redhead NSW 2290,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9LVRNBVZENGPROJECTS,11480,NaN,"50 Fitzroy Street_Carrington, 2294",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9ENWOUWMENGINEERING,11479,NaN,Level 18 Darling Park_2/201 Sussex Street_ Syd...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,CLQRGPUI,370,NaN,"48 Toh Guan Road East_#04-118, Enterprise Hub_...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,RUXHWWENGINEERING,10645,NaN,National Parks and Wildlife Service_Locked Bag...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,GYUFABBUILDING,4633,NaN,(T/As Lendlease Bouygues Joint Venture) _T/As ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,LTITRKTRZFQSERVICES,1175,NaN,Locked Bag 18_Port Melbourne VIC 3207,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Post Code, State and Suburb

In [38]:
# testing if there's any row not in the pattern

Billing_IDs_To_Exclude = []

error_reason = ''

states = ['NSW', 'VIC', 'QLD', 'SA', 'WA', 'TAS', 'ACT', 'NT']

for index, row in df.iterrows():
    
    customer_ID = row['ID']
    
    parts = row['Billing_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break
    
    
    #tests if the lenght of the Billing address is less than 2 lines, if yes, the address is not complete and it skips the rest of the code
    
    if len(parts) < 2:
        Billing_IDs_To_Exclude.append(customer_ID)
        error_reason += 'Address is not complete. It should have at least 2 lines!\n'
        
        new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
        
        Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
        
        error_reason = ''
        
        continue
        
    # if no, it tests if the last line, which should be the suburb details, have at least 3 elements, if yes, it gets the state and the code continues
    # case it's less than 3 elements, it skips the rest of the code 
    
    else:

        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are
        
        if len(suburb_details) > 2:
            state = suburb_details[-2].upper().strip(',').strip('`').strip('.')
        else:
            # Handle the case when the length is less than 3
            state = ''
            error_reason += 'Wrong Format. Please make sure the format looks like:\n{Address1}\n{Address2}\n{StreetNumber} {StreetName} {StreetType}\n{Suburb} {State} {Postcode}\n'
            Billing_IDs_To_Exclude.append(customer_ID)
            
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
            Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
        
            error_reason = ''
        
            continue
            
           #----------------------------------------------------------------------------------------------------------------------------------------------------------# 
            

        post_code = suburb_details[-1].strip(',').strip('`').strip().strip('.') # the post code is normally the last word in the last line

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        # Check if the post_code looks like 4 numbers
        is_post_code_valid = re.match(r'^\d{4}$', post_code) is not None

        # Check if the state looks like 3 or 2 letters
        is_state_valid = re.match(r'^[A-Za-z]{2,3}$', state) is not None
            
        
        if is_post_code_valid == False:
            
            if isinstance(post_code, int):
                error_reason += 'Postcode invalid. It should be 4 numbers.\n'
                Billing_IDs_To_Exclude.append(customer_ID)
                
                
            if not any(re.match(r'^\d{4}$', str(item)) is not None for item in suburb_details):
                error_reason += 'Postcode Missing.\n'
                Billing_IDs_To_Exclude.append(customer_ID)
                
                if not any(state in suburb_details for state in states):
                    error_reason += 'State Missing.\n'
                    
                    
                new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                        'ID': customer_ID,
                                                        'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                        'Billing Error Type': error_reason
                            }])

                Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)

                error_reason = ''
                
                continue

                
            else:
                
                if not any(state in suburb_details for state in states):
                    error_reason += 'State Missing.\n'
                    Billing_IDs_To_Exclude.append(customer_ID)
                    
                    new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
                    Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
                
                    error_reason = ''
                    
                    continue
                else:
                    error_reason += 'Wrong Format. Please make sure the format looks like:\n{Address1}\n{Address2}\n{StreetNumber} {StreetName} {StreetType}\n{Suburb} {State} {Postcode}\n'
                    Billing_IDs_To_Exclude.append(customer_ID)
                    
                    new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
                    Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
                
                    error_reason = ''
                    
                    continue
                    
                    
        
        if is_state_valid == False:
            
            if not any(state in suburb_details for state in states):
                error_reason += 'State Missing.\n'
                Billing_IDs_To_Exclude.append(customer_ID)
            else:
                Billing_IDs_To_Exclude.append(customer_ID)
                error_reason += 'State invalid. It should be 2 or 3 characters.\n'
                
                
        if suburb == '':
            error_reason += 'Suburb is Missing.'
            Billing_IDs_To_Exclude.append(customer_ID)

            
        if is_post_code_valid is False or is_state_valid is False or suburb == '':
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
            Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
                
        error_reason = ''

In [39]:
Billing_Error_Report

,Customer_Code,ID,Billing_Address,Billing Error Type
0,9LVRNBVZENGPROJECTS,11480,"50 Fitzroy Street\nCarrington, 2294",Wrong Format. Please make sure the format look...
1,9ENWOUWMENGINEERING,11479,Level 18 Darling Park\n2/201 Sussex Street\n S...,Wrong Format. Please make sure the format look...
2,9DLLZHAWIBJMAEYSPROJECTS,11470,Westlake QLD 4074\n,Address is not complete. It should have at lea...
3,9IUJVKPENGINEERING,11469,Contructors (Aust) Pty Ltd\nTBA\n,Wrong Format. Please make sure the format look...
4,9FOHAIWOQ,11438,Level 20\n1 Market Street\nSydney NSW 2000 Aus...,Wrong Format. Please make sure the format look...
5,GUEGGMIBDNABUILDING,10579,C/- Ticketmaster Australiasia\nLevel 5 – 364 L...,Wrong Format. Please make sure the format look...
6,9CBTYRQNXFRJICEWPLUMBING,10324,800 Feld Way\nPalmetto Florida USA 34221,Postcode Missing.\nState Missing.\n
7,9TKFSNBUILDING,10252,SA 5044,Address is not complete. It should have at lea...
8,9DRPNOLZENGINEERING,10171,"Lot 5, D'aguilar Highway\nMoore 4314 QLD",Wrong Format. Please make sure the format look...
9,9PCHUNK,9978,CRSI Ni Ltd\n136 Torman Road\nCarrickmore Ireland,Wrong Format. Please make sure the format look...


In [40]:
# Firt, we separate the Suburb, Post Code and State and put them into the new dataset

# The pattern we are looking for is: Condell Park NSW 2200 (Suburb State Postcode)

for index, row in df.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Billing_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Billing_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are

        post_code = suburb_details[-1].strip(',').strip('`').strip().strip('.') # the post code is normally the last word in the last line

        state = suburb_details[-2].upper().strip(',').strip('`').strip('.') # the state is normally the second to last

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        billing_address = '\n'.join(parts[:-1])

        billing_address += f'\n{suburb} {state} {post_code}'

        billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_Postcode'] = post_code
        billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_State'] = state
        billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_Suburb'] = suburb
        billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_Address'] = billing_address


#### Street Number and Street Name

In [41]:
# testing if there's any row not in the pattern

error_reason = ''

for index, row in df.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Billing_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Billing_Address'].strip('_').strip('.').split('_')

        street_details = parts[-2] # geting the second group of the split

        last_index = find_last_number_index(street_details) # geting the index of the last number, which represents the street number

        street_no = street_details[:last_index+1].strip(',').strip('.') # street no is everything until the index of the last number

        street_name = street_details[last_index+1:] # street name starts right after the last number

        street_name = street_name.strip(',').strip('`').strip().strip('.').title()
        
        if street_no == '':
            
            if not ("Cnr" in street_details or "Corner" in street_details):
                Billing_IDs_To_Exclude.append(customer_ID)
                error_reason += 'No street number found!\n'
            else:
                street_no = 'NULL'
            
        if street_name == '':
            Billing_IDs_To_Exclude.append(customer_ID)
            error_reason += 'No street name found!.\n'
            
            
        if street_no == '' or street_name == '':
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
            Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
                
        error_reason = ''

In [42]:
# Second, we get the street detail, that is normally the second group of the split

for index, row in df.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Billing_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Billing_Address'].strip('_').strip('.').split('_')

        street_details = parts[-2] # geting the second group of the split

        last_index = find_last_number_index(street_details) # geting the index of the last number, which represents the street number

        street_no = street_details[:last_index+1].strip(',').strip('.') # street no is everything until the index of the last number

        street_name = street_details[last_index+1:] # street name starts right after the last number

        street_name = street_name.strip(',').strip('`').strip().strip('.').strip(',').title()

        billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_StreetNo'] = street_no
        billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_StreetName'] = street_name

#### Billing Address 2

In [43]:
# Third, we get what we have in the third group for those who have something and put in the Billing Address 2 column
for index, row in df.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Billing_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Billing_Address'].strip('_').strip('.').split('_')

        trading_as = []
        adress2 = []

        if len(parts) > 2:

            for part in parts[:-2]:
                if part.lower().startswith("t/a") or part.lower().startswith("t/as"):
                    trading_as.append(part.strip().strip(',').strip('.'))
                else:
                    adress2.append(part.strip().strip(',').strip('.'))

            if len(adress2) > 0:
                billing_address2 = '\n'.join(adress2) # geting the second group of the split
                billing_address2 = billing_address2.strip('\n').strip('.')
                billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_Address2'] = billing_address2

            if len(trading_as) > 0:
                billing_ta = '\n'.join(trading_as)
                billing_ta = billing_ta.strip().strip(',').strip('.')
                billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_Customisable_Value'] = billing_ta
                billing_ta = None

#### Updating the Billing Address

In [44]:
for index, row in df.iterrows():
    
    customer_ID = row['ID']

    if customer_ID in Billing_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Billing_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are 

        post_code = suburb_details[-1] # the post code is normally the last word in the last line

        state = suburb_details[-2].upper() # the state is normally the second to last

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        billing_address = '\n'.join(parts[:-1])

        billing_address += f'\n{suburb} {state} {post_code}'

        billing_first_group.loc[billing_first_group['ID'] == customer_ID, 'Billing_Address'] = billing_address

In [45]:
billing_first_group = billing_first_group[~billing_first_group['ID'].isin(Billing_IDs_To_Exclude)]

billing_first_group

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2
0,9NBRZAGLJDHKHFBUILDING,11551,NaN,"Suite 4, Level 1, 374 Church Street \nPARRAMAT...","Suite 4, Level 1, 374",Church Street,NaN,PARRAMATTA,NSW,2150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9UNFBFE,11505,NaN,"Suite 802, Level 8, 24 Montgomery Street \nKOG...","Suite 802, Level 8, 24",Montgomery Street,NaN,KOGARAH,NSW,2217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9NPMZBPLUMBING,11334,NaN,88 Dobroyd Drive \nELIZABETH DRIVE NSW 2171,88,Dobroyd Drive,NaN,ELIZABETH DRIVE,NSW,2171,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,MKGCMMEBAEIFSHB,11307,NaN,145A Hutton Road \nTHE ENTRANCE NORTH NSW 2261,145A,Hutton Road,NaN,THE ENTRANCE NORTH,NSW,2261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,INNRSECW,11283,NaN,P/L & GS Engineering & Constructions Australia...,"Level 1, 436-438",Johnston Street,NaN,ABBOTSFORD,VIC,3067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,BTMZFCGNBUILDING,10974,NaN,Ground Floor Building 24 270 Ferntree Gully Ro...,Ground Floor Building 24 270,Ferntree Gully Road,NaN,NOTTING HILL,VIC,3168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,9ISWLNDMKQPXHMKXPROJECTS,10718,NaN,Cnr Stockton & Dredge Ave\nMOOREBANK NSW 2170,,Cnr Stockton & Dredge Ave,NaN,MOOREBANK,NSW,2170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,RTHQFYHWSTAEULZPROJECTS,10689,NaN,Suite 1601 Level 16 100 William Street\nWOOLLO...,Suite 1601 Level 16 100,William Street,NaN,WOOLLOOMOOLOO,NSW,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,XSZNYDXNBUILDING,10547,NaN,Ferrovial Agroman (Australia) Pty Ltd\nT/As Ne...,Level 9/32,Turbot Street,NaN,BRISBANE,QLD,4000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,XQSXTNKDYLCBUILDING,8853,NaN,Utilities P/L & Programmed Facility Management...,Level 4 11,Murray Rose Avenue,NaN,SYDNEY OLYMPIC PARK,NSW,2127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Second Group: PO Box and Locked Bag

In [47]:
# Establish the connection with Windows authentication

#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database

#cursor = conn.cursor()

select_billing_second = f"""

WITH UpdateSelection AS (

SELECT DISTINCT [Source_Id]
FROM [sample_database].[dbo].[VA_DBT_Master]
WHERE [Audited_Field] IN ('Billing_Address') 
AND Operation IN ('U', 'I')
AND Audit_DateTime > '{last_finish_date}'
AND Audit_User NOT LIKE 'TableauAdminAccess'

AND [After_Value] IS NOT NULL AND [After_Value] NOT LIKE 'TBA%' AND

([After_Value]  LIKE '%Locked Bag%' OR [After_Value] LIKE '%PO BOX%' OR [After_Value] LIKE '%P.O. BOX%' OR [After_Value] LIKE '%P.O BOX%'
 OR [After_Value] LIKE '%P O Box%' OR [After_Value] LIKE '%Private Bag%')

)

SELECT     
    [Customer_Code],
    [ID],
    REPLACE([Billing_Address],CHAR(13)+CHAR(10),'_') AS [Billing_Address]
FROM [sample_database].[dbo].[DBT_Master]

WHERE

[Billing_Address] IS NOT NULL AND [Billing_Address] NOT LIKE 'TBA%' AND

([Billing_Address]  LIKE '%Locked Bag%' OR [Billing_Address] LIKE '%PO BOX%' OR [Billing_Address] LIKE '%P.O. BOX%' OR [Billing_Address] LIKE '%P.O BOX%'
 OR [Billing_Address] LIKE '%P O Box%' OR [Billing_Address] LIKE '%Private Bag%')

AND [ID] IN (SELECT DISTINCT [Source_Id]
            FROM UpdateSelection)

        """
#df2 = pd.read_sql_query(select_billing_second, conn)

#cursor.close()
#conn.close()

df2 = pd.read_excel("Billing Address - PO Boxes.xlsx")

df2

,Customer_Code,ID,Billing_Address
0,9KMCTUNTYTSERVICES,11485,GPO Box 4311_Melbourne Victoria 3001
1,PBPHYIPLUMBING,11405,PO Box 5485 _Wollongong NSW 2520
2,9WAHBNWPXZENGINEERING,11289,PO Box 1161 _Mountain Gate Vic 3165 _
3,9EMKTVVGFJZXYUNWBUILDING,10999,T/As Odyssey Constructions & Fitout_PO Box 717...
4,9BGRBF,10677,PO BOX 13268_George Street_Queensland QLD 4003
5,YIFIOTLKGHCYBENGINEERING,10273,PO Box 9109_South Yarra 3141
6,DCCGOOARUQFQ,10267,PO Box 130_Morphett Vale 5162
7,9EPDYNAUSERVICES,10209,PO Box 279_Batman 3058
8,SAMOQDVBTHXPROJECTS,10153,PO Box 1160_Altona Meadows 3028
9,TAXPTQPROJECTS,9625,Engineering Pty Ltd T/As CPB Ghella UGL JV_Att...


In [48]:
billing_second_group = pd.DataFrame(columns=columns)

billing_second_group['Customer_Code'] = df2['Customer_Code']
billing_second_group['ID'] = df2['ID']
billing_second_group['Billing_Address'] = df2['Billing_Address']

billing_second_group

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2
0,9KMCTUNTYTSERVICES,11485,NaN,GPO Box 4311_Melbourne Victoria 3001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PBPHYIPLUMBING,11405,NaN,PO Box 5485 _Wollongong NSW 2520,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9WAHBNWPXZENGINEERING,11289,NaN,PO Box 1161 _Mountain Gate Vic 3165 _,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9EMKTVVGFJZXYUNWBUILDING,10999,NaN,T/As Odyssey Constructions & Fitout_PO Box 717...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9BGRBF,10677,NaN,PO BOX 13268_George Street_Queensland QLD 4003,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,YIFIOTLKGHCYBENGINEERING,10273,NaN,PO Box 9109_South Yarra 3141,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DCCGOOARUQFQ,10267,NaN,PO Box 130_Morphett Vale 5162,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9EPDYNAUSERVICES,10209,NaN,PO Box 279_Batman 3058,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SAMOQDVBTHXPROJECTS,10153,NaN,PO Box 1160_Altona Meadows 3028,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TAXPTQPROJECTS,9625,NaN,Engineering Pty Ltd T/As CPB Ghella UGL JV_Att...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Post Code, State and Suburb

In [49]:
# testing if there's any row not in the pattern

error_reason = ''


for index, row in df2.iterrows():
    
    customer_ID = row['ID']
    
    ## attention with the ID 7112
    
    parts = row['Billing_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break
    
    #tests if the lenght of the Billing address is less than 2 lines, if yes, the address is not complete and it skips the rest of the code
    
    if len(parts) < 2:
        Billing_IDs_To_Exclude.append(customer_ID)
        error_reason += 'Address is not complete. It should have at least 2 lines!\n'
        
        new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
        
        Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
        
        error_reason = ''
        
        continue
        
    # if no, it tests if the last line, which should be the suburb details, have at least 3 elements, if yes, it gets the state and the code continues
    # case it's less than 3 elements, it skips the rest of the code 
        
    else:
    
        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are 
        
        if len(suburb_details) > 2:
            state = suburb_details[-2].upper().strip(',').strip('`').strip('.')
        else:
            # Handle the case when the length is less than 3
            state = ''
            error_reason += 'Wrong Format. Please make sure the format looks like:\n{Address1}\n{Address2}\n{StreetNumber} {StreetName} {StreetType}\n{Suburb} {State} {Postcode}\n'
            Billing_IDs_To_Exclude.append(customer_ID)
            
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
            Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
        
            error_reason = ''
        
            continue
            
           #----------------------------------------------------------------------------------------------------------------------------------------------------------# 
            

        post_code = suburb_details[-1].strip(',').strip('`').strip().strip('.') # the post code is normally the last word in the last line

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        # Check if the post_code looks like 4 numbers
        is_post_code_valid = re.match(r'^\d{4}$', post_code) is not None

        # Check if the state looks like 3 or 2 letters
        is_state_valid = re.match(r'^[A-Za-z]{2,3}$', state) is not None
        
    
        keywords_to_exclude = ['Locked Bag', 'PO BOX', 'P.O. BOX', 'P.O BOX', 'P O Box', 'Private Bag']
            
        if any(keyword.upper() in suburb for keyword in keywords_to_exclude):
            Billing_IDs_To_Exclude.append(customer_ID)
            error_reason += 'Suburb contains address info.'
        
        if is_post_code_valid == False:
            
            Billing_IDs_To_Exclude.append(customer_ID)
            if isinstance(post_code, int):
                error_reason += 'Postcode invalid. It should be 4 numbers.\n'
                Billing_IDs_To_Exclude.append(customer_ID)
                
                
            if not any(re.match(r'^\d{4}$', str(item)) is not None for item in suburb_details):
                error_reason += 'Postcode Missing.\n'
                Billing_IDs_To_Exclude.append(customer_ID)
                
                if not any(state in suburb_details for state in states):
                    error_reason += 'State Missing.\n'
                    
                    
                new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                        'ID': customer_ID,
                                                        'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                        'Billing Error Type': error_reason
                            }])

                Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)

                error_reason = ''
                
                continue

                
            else:
                
                if not any(state in suburb_details for state in states):
                    error_reason += 'State Missing.\n'
                    Billing_IDs_To_Exclude.append(customer_ID)
                    
                    new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
                    Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
                
                    error_reason = ''
                    
                    continue
                else:
                    error_reason += 'Wrong Format. Please make sure the format looks like:\n{Address1}\n{Address2}\n{StreetNumber} {StreetName} {StreetType}\n{Suburb} {State} {Postcode}\n'
                    Billing_IDs_To_Exclude.append(customer_ID)
                    
                    new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
                    Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
                
                    error_reason = ''
                    
                    continue
                    
                    
        
        if is_state_valid == False:
            if not any(state in suburb_details for state in states):
                error_reason += 'State Missing.\n'
                Billing_IDs_To_Exclude.append(customer_ID)
            else:
                Billing_IDs_To_Exclude.append(customer_ID)
                error_reason += 'State invalid. It should be 2 or 3 characters.\n'
            
        
        if suburb == '':
            error_reason += 'Suburb is Missing.'
            Billing_IDs_To_Exclude.append(customer_ID)
        
            
        if is_post_code_valid is False or is_state_valid is False or suburb == '' or any(keyword.upper() in suburb for keyword in keywords_to_exclude):
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
            Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
                
        error_reason = ''

In [50]:
# Firt, we separate the Suburb, Post Code and State and put them into the new dataset

# The pattern we are looking for is: Condell Park NSW 2200 (Suburb State Post Code)

for index, row in df2.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Billing_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Billing_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are 

        post_code = suburb_details[-1].strip(',').strip('`').strip().strip('.') # the post code is normally the last word in the last line

        state = suburb_details[-2].upper().strip(',').strip('`').strip('.') # the state is normally the second to last

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        billing_second_group.loc[billing_second_group['ID'] == customer_ID, 'Billing_Postcode'] = post_code
        billing_second_group.loc[billing_second_group['ID'] == customer_ID, 'Billing_State'] = state
        billing_second_group.loc[billing_second_group['ID'] == customer_ID, 'Billing_Suburb'] = suburb

#### Billing Address 2

It was the billing address 2 the field that would receive the values, but after some problems in converting quotes in Salesforce, this field was changed to Billing_StreetName.

Reffer to this change in the TASK# 002370 - Split Billing Address - PO Box Cases (Baseplan)

In [51]:
for index, row in df2.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Billing_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Billing_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        trading_as = []
        adress2 = []
        streetname = []

        for part in parts[:-1]:
                if part.lower().startswith("t/a") or part.lower().startswith("t/as"):
                    trading_as.append(part.strip().strip(',').strip('.'))
                elif re.search(r'\bbox\b|\bbag\b', part, re.IGNORECASE):
                    streetname.append(part.strip().strip(',').strip('.'))
                else:                       
                    adress2.append(part.strip().strip(',').strip('.'))

        if len(adress2) > 0:
            billing_address2 = '\n'.join(adress2) # geting the second group of the split
            billing_address2 = billing_address2.strip('\n').strip().strip(',').strip('.')
            billing_second_group.loc[billing_second_group['ID'] == customer_ID, 'Billing_Address2'] = billing_address2
        
        if len(streetname) > 0:
            streetName = '\n'.join(streetname) # geting the second group of the split
            streetName.strip('\n').strip().strip(',').strip('.')            
            billing_second_group.loc[billing_second_group['ID'] == customer_ID, 'Billing_StreetName'] = streetname

        if len(trading_as) > 0:
            billing_ta = '\n'.join(trading_as)
            billing_ta = billing_ta.strip('\n').strip().strip(',').strip('.')
            billing_second_group.loc[billing_second_group['ID'] == customer_ID, 'Billing_Customisable_Value'] = billing_ta
            billing_ta = None

#### Updating the Billing Address

In [52]:
for index, row in df2.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Billing_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Billing_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are

        post_code = suburb_details[-1].strip('.') # the post code is normally the last word in the last line

        state = suburb_details[-2].upper() # the state is normally the second to last

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        billing_address = '\n'.join(parts[:-1])

        billing_address += f'\n{suburb} {state} {post_code}'

        billing_second_group.loc[billing_second_group['ID'] == customer_ID, 'Billing_Address'] = billing_address

In [53]:
billing_second_group = billing_second_group[~billing_second_group['ID'].isin(Billing_IDs_To_Exclude)]

billing_second_group

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2
1,PBPHYIPLUMBING,11405,NaN,PO Box 5485 \nWOLLONGONG NSW 2520,NaN,PO Box 5485,NaN,WOLLONGONG,NSW,2520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9WAHBNWPXZENGINEERING,11289,NaN,PO Box 1161 \nMOUNTAIN GATE VIC 3165,NaN,PO Box 1161,NaN,MOUNTAIN GATE,VIC,3165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9EMKTVVGFJZXYUNWBUILDING,10999,NaN,T/As Odyssey Constructions & Fitout\nPO Box 71...,NaN,PO Box 7179,NaN,WETHERILL PARK BC,NSW,2164,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9BGRBF,10677,NaN,PO BOX 13268\nGeorge Street\nQUEENSLAND QLD 4003,NaN,PO BOX 13268,NaN,QUEENSLAND,QLD,4003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TAXPTQPROJECTS,9625,NaN,Engineering Pty Ltd T/As CPB Ghella UGL JV\nAt...,NaN,PO Box 7261,NaN,ALEXANDRIA,NSW,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,ZWWGFVULCXOHSERVICES,9384,NaN,PO Box 86\nPOORAKA SA 5095,NaN,PO Box 86,NaN,POORAKA,SA,5095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,FOSTLYRCPROJECTS,8954,NaN,PO Box 567\nCARINA HEIGHTS QLD 4152,NaN,PO Box 567,NaN,CARINA HEIGHTS,QLD,4152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,LDONRWFLJVQKZQCONSTRUCTION,8683,NaN,Laing O'Rourke Australia Group Pty Ltd\nAccoun...,NaN,GPO Box 5094,NaN,BRISBANE,QLD,4001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,UECRDNBBAYENGINEERING,6935,NaN,PO Box 3355\nDARRA QLD 4076,NaN,PO Box 3355,NaN,DARRA,QLD,4076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NYULZQAGEEKPY,6519,NaN,P.O Box 1410\nBEENLEIGH QLD 4207,NaN,P.O Box 1410,NaN,BEENLEIGH,QLD,4207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Delivery Address

### First Group: the standard data

In [55]:
# Establish the connection with Windows authentication

#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database
#cursor = conn.cursor()


select_delivery_first = f"""

WITH UpdateSelection AS (

SELECT DISTINCT [Source_Id]
FROM [sample_database].[dbo].[VA_DBT_Master]
WHERE [Audited_Field] IN ('Delivery_Address') 
AND Operation IN ('U', 'I') 
AND Audit_DateTime > '{last_finish_date}' 
AND Audit_User NOT LIKE 'TableauAdminAccess'

AND [After_Value] IS NOT NULL AND [After_Value] NOT LIKE 'TBA%' AND NOT

([After_Value]  LIKE '%Locked Bag%' OR [After_Value] LIKE '%PO BOX%' OR [After_Value] LIKE '%P.O. BOX%' OR [After_Value] LIKE '%P.O BOX%'
 OR [After_Value] LIKE '%P O Box%' OR [After_Value] LIKE '%Private Bag%')

)

SELECT     
    [Customer_Code],
    [ID],
    REPLACE([Delivery_Address],CHAR(13)+CHAR(10),'_') AS [Delivery_Address]
FROM [sample_database].[dbo].[DBT_Master]

WHERE 

[Delivery_Address] IS NOT NULL AND [Delivery_Address] NOT LIKE 'TBA%' AND NOT

([Delivery_Address]  LIKE '%Locked Bag%' OR [Delivery_Address] LIKE '%PO BOX%' OR [Delivery_Address] LIKE '%P.O. BOX%' OR [Delivery_Address] LIKE '%P.O BOX%'
 OR [Delivery_Address] LIKE '%P O Box%' OR [Delivery_Address] LIKE '%Private Bag%')

AND [ID] IN (SELECT DISTINCT [Source_Id]
            FROM UpdateSelection)
                    
"""

#df3 = pd.read_sql_query(select_delivery_first, conn)

#cursor.close()
#conn.close()

df3 = pd.read_excel("Delivery Address.xlsx")

df3

,Customer_Code,ID,Delivery_Address
0,PORTADELAIDECONSTRUC,11669,Cnr Jetty & Elder Road_Largs North SA 5016
1,9CRONULLAFERRIES,11660,"Public Wharf, Tonkin Street_Cronulla NSW 2230"
2,9AMTGRANDHOMES,11551,"Suite 4, Level 1, 374 Church Street _Parramatt..."
3,9BLUZONE,11505,"Suite 802, Level 8, 24 Montgomery Street _Koga..."
4,9HPOTS,11503,McFarlane Street Extended_Cessnock NSW 2325
...,...,...,...
215,9JECCONCRETING,160,27 Telopea Street_Punchbowl
216,9GROUPACONS,145,15 Erang Street _CARSS PARK 2221
217,9FORMPROJECTS,136,Bells Line Road_Kurmond NSW 2757
218,9BUILDABILITY,71,108/55 Jones Street_ULTIMO


In [56]:
delivery_first_group = pd.DataFrame(columns=columns)

delivery_first_group['Customer_Code'] = df3['Customer_Code']
delivery_first_group['ID'] = df3['ID']
delivery_first_group['Delivery_Address'] = df3['Delivery_Address']

delivery_first_group

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2
0,PORTADELAIDECONSTRUC,11669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Cnr Jetty & Elder Road_Largs North SA 5016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9CRONULLAFERRIES,11660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Public Wharf, Tonkin Street_Cronulla NSW 2230",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9AMTGRANDHOMES,11551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Suite 4, Level 1, 374 Church Street _Parramatt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9BLUZONE,11505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"Suite 802, Level 8, 24 Montgomery Street _Koga...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9HPOTS,11503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,McFarlane Street Extended_Cessnock NSW 2325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,9JECCONCRETING,160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,27 Telopea Street_Punchbowl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,9GROUPACONS,145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15 Erang Street _CARSS PARK 2221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,9FORMPROJECTS,136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Bells Line Road_Kurmond NSW 2757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,9BUILDABILITY,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,108/55 Jones Street_ULTIMO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Post Code, State and Suburb

In [57]:
# testing if there's any row not in the pattern

Delivery_IDs_To_Exclude = []

error_reason = ''

for index, row in df3.iterrows():
    
    customer_ID = row['ID']
    
    parts = str(row['Delivery_Address']).strip('_').strip('.').split('_') # spliting the string by _ that represents a line break
    
    
    #tests if the lenght of the Billing address is less than 2 lines, if yes, the address is not complete and it skips the rest of the code

    if len(parts) < 2:
        Delivery_IDs_To_Exclude.append(customer_ID)
        error_reason += 'Address is not complete. It should have at least 2 lines!\n'
        
        new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
        
        Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)
        
        error_reason = ''
        
        continue
        
    else:
    
        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are
        
        if len(suburb_details) >= 2:
            state = suburb_details[-2].upper().strip(',').strip('`').strip('.') # the state is normally the second to last
        else:
            # Handle the case when the length is less than 2
            state = ''
            error_reason += 'Wrong Format. Please make sure the format looks like:\n{Address1}\n{Address2}\n{StreetNumber} {StreetName} {StreetType}\n{Suburb} {State} {Postcode}\n'
            Delivery_IDs_To_Exclude.append(customer_ID)
            
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
            Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)
        
            error_reason = ''
        
            continue
            
           #----------------------------------------------------------------------------------------------------------------------------------------------------------# 

        post_code = suburb_details[-1].strip(',').strip('`').strip().strip('.').strip('.') # the post code is normally the last word in the last line

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        # Check if the post_code looks like 4 numbers
        is_post_code_valid = re.match(r'^\d{4}$', post_code) is not None

        # Check if the state looks like 3 or 2 letters
        is_state_valid = re.match(r'^[A-Za-z]{2,3}$', state) is not None
        
            
        if is_post_code_valid == False:
            if isinstance(post_code, int):
                error_reason += 'Postcode invalid. It should be 4 numbers.\n'
                Delivery_IDs_To_Exclude.append(customer_ID)
                
                
            if not any(re.match(r'^\d{4}$', str(item)) is not None for item in suburb_details):
                error_reason += 'Postcode Missing.\n'
                Delivery_IDs_To_Exclude.append(customer_ID)
                
                if not any(state in suburb_details for state in states):
                    error_reason += 'State Missing.\n'
                    
                    
                new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
                Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)

                error_reason = ''
                
                continue

                
            else:
                
                if not any(state in suburb_details for state in states):
                    error_reason += 'State Missing.\n'
                    Delivery_IDs_To_Exclude.append(customer_ID)
                    
                    new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
                    Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)

                    error_reason = ''
                    
                    continue
                    
                else:
                    error_reason += 'Wrong Format. Please make sure the format looks like:\n{Address1}\n{Address2}\n{StreetNumber} {StreetName} {StreetType}\n{Suburb} {State} {Postcode}\n'
                    Delivery_IDs_To_Exclude.append(customer_ID)
                    
                    new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
                    Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)

                    error_reason = ''
                    
                    continue
                    
        
        if is_state_valid == False:
            
            if not any(state in suburb_details for state in states):
                error_reason += 'State Missing.\n'
                Delivery_IDs_To_Exclude.append(customer_ID)
            else:
                Delivery_IDs_To_Exclude.append(customer_ID)
                error_reason += 'State invalid. It should be 2 or 3 characters.\n'
                
        
        if suburb == '':
            error_reason += 'Suburb is Missing.'
            Delivery_IDs_To_Exclude.append(customer_ID)
            
            
        if is_post_code_valid is False or is_state_valid is False or suburb == '':
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
            Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)
                
        error_reason = ''

In [58]:
# Firt, we separate the Suburb, Post Code and State and put them into the new dataset

# The pattern we are looking for is: Condell Park NSW 2200 (Suburb State Post Code)

for index, row in df3.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Delivery_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Delivery_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        suburb_details = parts[-1].strip('.').split() # the last part is usually where the Suburb, post code and state are 

        post_code = suburb_details[-1].strip(',').strip('`').strip().strip('.') # the post code is normally the last word in the last line

        state = suburb_details[-2].upper().strip(',').strip('`').strip('.') # the state is normally the second to last

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        delivery_first_group.loc[delivery_first_group['ID'] == customer_ID, 'Delivery_Postcode'] = post_code
        delivery_first_group.loc[delivery_first_group['ID'] == customer_ID, 'Delivery_State'] = state
        delivery_first_group.loc[delivery_first_group['ID'] == customer_ID, 'Delivery_Suburb'] = suburb

#### Street Number and Street Name

In [59]:
# testing if the street number is empty, in case it if, it's not in the pattern

error_reason = ''

for index, row in df3.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Delivery_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Delivery_Address'].strip('_').strip('.').split('_')

        street_details = parts[-2] # geting the second group of the split

        last_index = find_last_number_index(street_details) # geting the index of the last number, which represents the street number

        street_no = street_details[:last_index+1].strip(',').strip('.') # street no is everything until the index of the last number

        street_name = street_details[last_index+1:] # street name starts right after the last number

        street_name = street_name.strip(',').strip('`').strip().strip('.').title()

        if street_no == '':
            
            if not ("Cnr" in street_details or "Corner" in street_details):
                Delivery_IDs_To_Exclude.append(customer_ID)
                error_reason += 'No street number found!\n'
            else:
                street_no = 'NULL'

        if street_name == '':
            Delivery_IDs_To_Exclude.append(customer_ID)
            error_reason += 'No street name found!\n'


        if street_no == '' or street_name == '':
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])

            Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)

        error_reason = ''

In [60]:
# Second, we get the street detail, that is normally the second group of the split

for index, row in df3.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Delivery_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Delivery_Address'].strip('_').strip('.').split('_')

        street_details = parts[-2] # geting the second group of the split

        last_index = find_last_number_index(street_details) # geting the index of the last number, which represents the street number

        street_no = street_details[:last_index+1].strip(',').strip('.') # street no is everything until the index of the last number

        street_name = street_details[last_index+1:] # street name starts right after the last number

        street_name = street_name.strip(',').strip('`').strip().strip('.')

        delivery_first_group.loc[delivery_first_group['ID'] == customer_ID, 'Delivery_StreetNo'] = street_no
        delivery_first_group.loc[delivery_first_group['ID'] == customer_ID, 'Delivery_StreetName'] = street_name

#### Billing Address 2

In [63]:
# Third, we get what we have in the tird group for those who have something and put in the Billing Address 2 column
for index, row in df3.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Delivery_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Delivery_Address'].strip('_').strip('.').split('_')

        trading_as = []
        adress2 = []

        if len(parts) > 2:

            for part in parts[:-2]:
                if part.lower().startswith("t/a") or part.lower().startswith("t/as"):
                    trading_as.append(part.strip().strip(',').strip('.'))
                else:
                    adress2.append(part.strip().strip(',').strip('.'))


            if len(adress2) > 0:
                delivery_address2 = '\n'.join(adress2) # geting the second group of the split
                delivery_address2 = delivery_address2.strip('\n').strip('.')
                delivery_first_group.loc[delivery_first_group['ID'] == customer_ID, 'Delivery_Address2'] = delivery_address2

            if len(trading_as) > 0:
                delivery_ta = '\n'.join(trading_as)
                delivery_ta = delivery_ta.strip().strip(',')
                delivery_first_group.loc[delivery_first_group['ID'] == customer_ID, 'Delivery_Customisable_Value'] = delivery_ta
                delivery_ta = None

#### Updating the Delivery Address

In [64]:
for index, row in df3.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Delivery_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Delivery_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are 

        post_code = suburb_details[-1].strip().strip(',').strip('.') # the post code is normally the last word in the last line

        state = suburb_details[-2].upper().strip().strip(',').strip('.') # the state is normally the second to last

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        delivery_address = '\n'.join(parts[:-1])

        delivery_address += f'\n{suburb} {state} {post_code}'

        delivery_first_group.loc[delivery_first_group['ID'] == customer_ID, 'Delivery_Address'] = delivery_address

In [65]:
delivery_first_group = delivery_first_group[~delivery_first_group['ID'].isin(Delivery_IDs_To_Exclude)]

delivery_first_group[['Customer_Code', 'ID', 'Delivery_Address', 'Delivery_Customisable_Value']]

,Customer_Code,ID,Delivery_Address,Delivery_Customisable_Value
0,PORTADELAIDECONSTRUC,11669,Cnr Jetty & Elder Road\nLARGS NORTH SA 5016,NaN
2,9AMTGRANDHOMES,11551,"Suite 4, Level 1, 374 Church Street \nPARRAMAT...",NaN
3,9BLUZONE,11505,"Suite 802, Level 8, 24 Montgomery Street \nKOG...",NaN
7,9PORTWARATAHCOAL,11486,Cnr Port Waratah Drive and Elizabeth Street\nC...,NaN
15,9KNOWITCONS,11426,4/32 Liney Avenue\nCLEMTON PARK NSW 2206,NaN
...,...,...,...,...
200,BROADCAST,596,C/-Broadcast Australia Transmitting Station\nF...,NaN
202,BALLYHOOLY,554,2388 Silverdale Road\nSILVERDALE NSW 2752,NaN
206,ICMP,371,Cnr Jones Bay & Pirrama Roads\nPYRMONT NSW 2009,NaN
209,9QUACKHOU,327,Cnr Stockton & Dredge Avenues\nMOOREBANK NSW 2170,NaN


### Second Group: PO Box and Locked Bag

In [66]:
# Establish the connection with Windows authentication

#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database

#cursor = conn.cursor()


select_delivery_second = f"""

WITH UpdateSelection AS (

SELECT DISTINCT [Source_Id]
FROM [sample_database].[dbo].[VA_DBT_Master]
WHERE [Audited_Field] IN ('Delivery_Address') 
AND Operation IN ('U', 'I')
AND Audit_DateTime > '{last_finish_date}'
AND Audit_User NOT LIKE 'TableauAdminAccess'

AND [After_Value] IS NOT NULL AND [After_Value] NOT LIKE 'TBA%' AND

([After_Value]  LIKE '%Locked Bag%' OR [After_Value] LIKE '%PO BOX%' OR [After_Value] LIKE '%P.O. BOX%' OR [After_Value] LIKE '%P.O BOX%'
 OR [After_Value] LIKE '%P O Box%' OR [After_Value] LIKE '%Private Bag%')

)

SELECT     
    [Customer_Code],
    [ID],
    REPLACE([Delivery_Address],CHAR(13)+CHAR(10),'_') AS [Delivery_Address]
FROM [sample_database].[dbo].[DBT_Master]

WHERE 

[Delivery_Address] IS NOT NULL AND [Delivery_Address] NOT LIKE 'TBA%' AND

([Delivery_Address]  LIKE '%Locked Bag%' OR [Delivery_Address] LIKE '%PO BOX%' OR [Delivery_Address] LIKE '%P.O. BOX%' OR [Delivery_Address] LIKE '%P.O BOX%'
 OR [Delivery_Address] LIKE '%P O Box%' OR [Delivery_Address] LIKE '%Private Bag%')

AND [ID] IN (SELECT DISTINCT [Source_Id]
            FROM UpdateSelection)
            
"""

#df4 = pd.read_sql_query(select_delivery_second, conn)

#cursor.close()
#conn.close()

df4 = pd.read_excel("Delivery Address - PO Boxes.xlsx")

df4

,Customer_Code,ID,Delivery_Address


In [67]:
delivery_second_group = pd.DataFrame(columns=columns)

delivery_second_group['Customer_Code'] = df4['Customer_Code']
delivery_second_group['ID'] = df4['ID']
delivery_second_group['Delivery_Address'] = df4['Delivery_Address']

delivery_second_group

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2


#### Post Code, State and Suburb

In [68]:
# testing if there's any row not in the pattern

error_reason = ''

for index, row in df4.iterrows():
    
    customer_ID = row['ID']
    
    parts = row['Delivery_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break
    
    if len(parts) < 2:
        Delivery_IDs_To_Exclude.append(customer_ID)
        error_reason += 'Address is not complete. It should have at least 2 lines!\n'
        
        new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
        
        Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)
        
        error_reason = ''
        
        continue
        
    else:
    
        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are 
        
        if len(suburb_details) >= 2:
            state = suburb_details[-2].upper().strip(',').strip('`').strip('.') # the state is normally the second to last
        else:
            # Handle the case when the length is less than 2
            state = ''
            error_reason += 'Wrong Format. Please make sure the format looks like:\n{Address1}\n{Address2}\n{StreetNumber} {StreetName} {StreetType}\n{Suburb} {State} {Postcode}\n'
            Delivery_IDs_To_Exclude.append(customer_ID)
            
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
            Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)
        
            error_reason = ''
        
            continue
            
           #----------------------------------------------------------------------------------------------------------------------------------------------------------# 

        post_code = suburb_details[-1].strip(',').strip('`').strip().strip('.') # the post code is normally the last word in the last line

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        # Check if the post_code looks like 4 numbers
        is_post_code_valid = re.match(r'^\d{4}$', post_code) is not None

        # Check if the state looks like 3 or 2 letters
        is_state_valid = re.match(r'^[A-Za-z]{2,3}$', state) is not None
        
        keywords_to_exclude = ['Locked Bag', 'PO BOX', 'P.O. BOX', 'P.O BOX', 'P O Box', 'Private Bag']
            
            
        if any(keyword.upper() in suburb for keyword in keywords_to_exclude):
            Delivery_IDs_To_Exclude.append(customer_ID)
            error_reason += 'Suburb contains address words.'


        
        if is_post_code_valid == False:
            if isinstance(post_code, int):
                error_reason += 'Postcode invalid. It should be 4 numbers.\n'
                Delivery_IDs_To_Exclude.append(customer_ID)
                
                
            if not any(re.match(r'^\d{4}$', str(item)) is not None for item in suburb_details):
                error_reason += 'Postcode Missing.\n'
                Delivery_IDs_To_Exclude.append(customer_ID)
                
                if not any(state in suburb_details for state in states):
                    error_reason += 'State Missing.\n'
                    
                    
                new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
                Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)

                error_reason = ''
                
                continue

                
            else:
                
                if not any(state in suburb_details for state in states):
                    error_reason += 'State Missing.\n'
                    Delivery_IDs_To_Exclude.append(customer_ID)
                    
                    new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
                    Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)

                    error_reason = ''
                    
                    continue
                    
                else:
                    error_reason += 'Wrong Format. Please make sure the format looks like:\n{Address1}\n{Address2}\n{StreetNumber} {StreetName} {StreetType}\n{Suburb} {State} {Postcode}\n'
                    Delivery_IDs_To_Exclude.append(customer_ID)
                    
                    new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
                    Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)

                    error_reason = ''
                    
                    continue
        
        
        
        if is_state_valid == False:
            
            if not any(state in suburb_details for state in states):
                error_reason += 'State Missing.\n'
                Delivery_IDs_To_Exclude.append(customer_ID)
            else:
                Delivery_IDs_To_Exclude.append(customer_ID)
                error_reason += 'State invalid. It should be 2 or 3 characters.\n'
            
        
        if suburb == '':
            error_reason += 'Suburb is Missing.'
            Delivery_IDs_To_Exclude.append(customer_ID)
        
            
        if is_post_code_valid is False or is_state_valid is False or suburb == '' or any(keyword.upper() in suburb for keyword in keywords_to_exclude):
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_ID,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
            Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)
                
        error_reason = ''

In [69]:
# Firt, we separate the Suburb, Post Code and State and put them into the new dataset

# The pattern we are looking for is: Condell Park NSW 2200 (Suburb State Post Code)

for index, row in df4.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Delivery_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Delivery_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are 

        post_code = suburb_details[-1].strip(',').strip('`').strip().strip('.') # the post code is normally the last word in the last line

        state = suburb_details[-2].upper().strip(',').strip('`').strip('.') # the state is normally the second to last

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        delivery_second_group.loc[delivery_second_group['ID'] == customer_ID, 'Delivery_Postcode'] = post_code
        delivery_second_group.loc[delivery_second_group['ID'] == customer_ID, 'Delivery_State'] = state
        delivery_second_group.loc[delivery_second_group['ID'] == customer_ID, 'Delivery_Suburb'] = suburb

#### Billing Address 2

It was the billing address 2 the field that would receive the values, but after some problems in converting quotes in Salesforce, this field was changed to Billing_StreetName.

Reffer to this change in the TASK# 002370 - Split Billing Address - PO Box Cases (Baseplan)

In [70]:
for index, row in df4.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Delivery_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Delivery_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        trading_as = []
        adress2 = []
        streetname = []

        for part in parts[:-1]:
                if part.lower().startswith("t/a") or part.lower().startswith("t/as"):
                    trading_as.append(part.strip().strip(',').strip('.'))
                elif re.search(r'\bbox\b|\bbag\b', part, re.IGNORECASE):
                    streetname.append(part.strip().strip(',').strip('.'))
                else:                       
                    adress2.append(part.strip().strip(',').strip('.'))

        if len(adress2) > 0:
            delivery_address2 = '\n'.join(adress2) # geting the second group of the split
            delivery_address2 = delivery_address2.strip('\n').strip('.')
            delivery_second_group.loc[delivery_second_group['ID'] == customer_ID, 'Delivery_Address2'] = delivery_address2
            
        if len(streetname) > 0:
            delivery_streetName = '\n'.join(streetname) # geting the second group of the split
            delivery_streetName.strip('\n').strip().strip(',').strip('.')
            delivery_second_group.loc[delivery_second_group['ID'] == customer_ID, 'Delivery_StreetName'] = streetname

        if len(trading_as) > 0:
            delivery_ta = '\n'.join(trading_as)
            delivery_ta = delivery_ta.strip().strip(',')
            delivery_second_group.loc[delivery_second_group['ID'] == customer_ID, 'Delivery_Customisable_Value'] = delivery_ta
            delivery_ta = None

#### Updating the Delivery Address

In [71]:
for index, row in df4.iterrows():
    
    customer_ID = row['ID']
    
    if customer_ID in Delivery_IDs_To_Exclude:
        continue
        
    else:
    
        parts = row['Delivery_Address'].strip('_').strip('.').split('_') # spliting the string by _ that represents a line break

        suburb_details = parts[-1].split() # the last part is usually where the Suburb, post code and state are 

        post_code = suburb_details[-1].strip().strip(',').strip('.') # the post code is normally the last word in the last line

        state = suburb_details[-2].upper().strip().strip(',').strip('.') # the state is normally the second to last

        suburb = ' '.join(suburb_details[:-2]).upper().strip(',').strip('.') # suburb is the rest, it can be 2 or more words, so we join them with a space

        delivery_address = '\n'.join(parts[:-1])

        delivery_address += f'\n{suburb} {state} {post_code}'

        delivery_second_group.loc[delivery_second_group['ID'] == customer_ID, 'Delivery_Address'] = delivery_address

In [72]:
delivery_second_group = delivery_second_group[~delivery_second_group['ID'].isin(Delivery_IDs_To_Exclude)]

delivery_second_group

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2


In [73]:
values_before_df = pd.concat([df, df2, df3, df4], ignore_index=True)

values_before_df

,Customer_Code,ID,Billing_Address,Delivery_Address
0,9NBRZAGLJDHKHFBUILDING,11551,"Suite 4, Level 1, 374 Church Street _Parramatt...",NaN
1,9UNFBFE,11505,"Suite 802, Level 8, 24 Montgomery Street _Koga...",NaN
2,9JZYZDTADNEKEACJ,11493,Saltspray Close_Redhead NSW 2290,NaN
3,9LVRNBVZENGPROJECTS,11480,"50 Fitzroy Street_Carrington, 2294",NaN
4,9ENWOUWMENGINEERING,11479,Level 18 Darling Park_2/201 Sussex Street_ Syd...,NaN
...,...,...,...,...
403,9JECCONCRETING,160,NaN,27 Telopea Street_Punchbowl
404,9GROUPACONS,145,NaN,15 Erang Street _CARSS PARK 2221
405,9FORMPROJECTS,136,NaN,Bells Line Road_Kurmond NSW 2757
406,9BUILDABILITY,71,NaN,108/55 Jones Street_ULTIMO


# Uniting the dataframes

In [77]:
# Establish the connection with Windows authentication

#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database

#cursor = conn.cursor()

select_all_ids = """
SELECT     
	[Customer_Code],
    [ID]
FROM [sample_database].[dbo].[DBT_Master]
ORDER BY ID
"""

#Addresses_Complete = pd.read_sql_query(select_all_ids, conn)

#cursor.close()
#conn.close()

Addresses_Complete = pd.read_excel("All Customers.xlsx")

for col in columns[2:]:
    Addresses_Complete[col] = None


Addresses_Complete

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2
0,9KMCTUNTYTSERVICES,11485,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,PBPHYIPLUMBING,11405,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,9WAHBNWPXZENGINEERING,11289,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,9EMKTVVGFJZXYUNWBUILDING,10999,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,9BGRBF,10677,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,R32SY39X77,160,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
301,I8K2LHX3VO,145,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
302,SEV3K1AXTE,136,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
303,49ZLTVKUB5,71,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [78]:
for index, row in Addresses_Complete.iterrows():
    
    customer_id = row['ID']
    
    # Find the corresponding row in the other DataFrames based on 'ID'
    billing_first_row = billing_first_group[billing_first_group['ID'] == customer_id]
    billing_second_row = billing_second_group[billing_second_group['ID'] == customer_id]
    delivery_first_row = delivery_first_group[delivery_first_group['ID'] == customer_id]
    delivery_second_row = delivery_second_group[delivery_second_group['ID'] == customer_id]
    
    # Check if there are matching rows in the DataFrames
    if not billing_first_row.empty:
        for column in columns[2:]:
            value = billing_first_row[column].values[0]
            if pd.isna(value):
                continue
            else:
                Addresses_Complete.at[index, column] = value

    if not billing_second_row.empty:
        for column in columns[2:]:
            value = billing_second_row[column].values[0]
            if pd.isna(value):
                continue
            else:
                Addresses_Complete.at[index, column] = value

    if not delivery_first_row.empty:
        for column in columns[2:]:
            value = delivery_first_row[column].values[0]
            if pd.isna(value):
                continue
            else:
                Addresses_Complete.at[index, column] = value

    if not delivery_second_row.empty:
        for column in columns[2:]:
            value = delivery_second_row[column].values[0]
            if pd.isna(value):
                continue
            else:
                Addresses_Complete.at[index, column] = value

In [79]:
# Dropping the line where all values are null
Addresses_Complete = Addresses_Complete.dropna(subset = Addresses_Complete.columns[2:], how='all')

Addresses_Complete.reset_index(drop = True, inplace = True)

Addresses_Complete

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,...,Delivery_ADR_Address_ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2
0,PBPHYIPLUMBING,11405,None,PO Box 5485 \nWOLLONGONG NSW 2520,None,PO Box 5485,None,WOLLONGONG,NSW,2520,...,None,"Suite 1, Level 6, 288 Forest Road \nHURSTVILL...","Suite 1, Level 6, 288",Forest Road,None,HURSTVILLE,NSW,2220,None,None
1,9WAHBNWPXZENGINEERING,11289,None,PO Box 1161 \nMOUNTAIN GATE VIC 3165,None,PO Box 1161,None,MOUNTAIN GATE,VIC,3165,...,None,"Suite 2, Level 11/555 Lonsdale Street \nMELBOU...","Suite 2, Level 11/555",Lonsdale Street,None,MELBOURNE,VIC,3000,None,None
2,9EMKTVVGFJZXYUNWBUILDING,10999,None,T/As Odyssey Constructions & Fitout\nPO Box 71...,None,PO Box 7179,None,WETHERILL PARK BC,NSW,2164,...,None,T/As Odyssey Constructions & Fitout\nSuite 1E ...,Suite 1E Level 1 1345,The Horsley Drive,None,WETHERILL PARK,NSW,2164,T/As Odyssey Constructions & Fitout,None
3,9BGRBF,10677,None,PO BOX 13268\nGeorge Street\nQUEENSLAND QLD 4003,None,PO BOX 13268,None,QUEENSLAND,QLD,4003,...,None,None,None,None,None,None,None,None,None,None
4,TAXPTQPROJECTS,9625,None,Engineering Pty Ltd T/As CPB Ghella UGL JV\nAt...,None,PO Box 7261,None,ALEXANDRIA,NSW,2016,...,None,Engineering Pty Ltd T/As CPB Ghella UGL JV\nLe...,Level 7 197-201,Coward Street,None,MASCOT,NSW,2020,None,Engineering Pty Ltd T/As CPB Ghella UGL JV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,A1MYY3VUE1,1024,None,None,None,None,None,None,None,None,...,None,Cnr Five Islands & Flinders Street\nPORT KEMBL...,,Cnr Five Islands & Flinders Street,None,PORT KEMBLA,NSW,2505,None,None
127,NSNVUW4LWO,781,None,None,None,None,None,None,None,None,...,None,Cnr of Kerr & Aero Roads\nINGLEBURN NSW 2565,,Cnr of Kerr & Aero Roads,None,INGLEBURN,NSW,2565,None,None
128,BKBHQFGT54,554,None,None,None,None,None,None,None,None,...,None,2388 Silverdale Road\nSILVERDALE NSW 2752,2388,Silverdale Road,None,SILVERDALE,NSW,2752,None,None
129,FH9YG6L9UT,327,None,None,None,None,None,None,None,None,...,None,Cnr Stockton & Dredge Avenues\nMOOREBANK NSW 2170,,Cnr Stockton & Dredge Avenues,None,MOOREBANK,NSW,2170,None,None


# Updating the database

This part gets the list of all suburbs from the system.

In [80]:
# Establish the connection with Windows authentication

#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database
#cursor = conn.cursor()

select_suburb_details = """
SELECT SUB.[Id]
      ,[SuburbName]
	  ,STA.[StateName]
      ,[StateID]
      ,[Postcode]
  FROM [sample_database].[dbo].[ADR_Suburb] SUB
	LEFT JOIN [sample_database].[dbo].[ADR_State] STA
		ON SUB.[StateID] = STA.[Id]
"""

#suburbs = pd.read_sql_query(select_suburb_details, conn)

#cursor.close()
#conn.close()

suburbs = pd.read_excel("Suburbs List.xlsx")

suburbs

,Id,SuburbName,StateName,StateID,Postcode
0,1,ACTON,ACT,1,2601
1,2,AINSLIE,ACT,1,2602
2,3,AMAROO,ACT,1,2914
3,4,ARANDA,ACT,1,2614
4,5,AUSTRALIAN NATIONAL UNIVERSITY,ACT,1,200
...,...,...,...,...,...
19651,16767,YOURDAMUNG LAKE,WA,8,6225
19652,19648,YUIN,WA,8,6635
19653,16768,YUNA,WA,8,6532
19654,16769,ZANTHUS,WA,8,6434


In [81]:
# Select the columns for billing information and 'Customer_Code' and 'ID'
billing_columns = ['Customer_Code', 'ID', 'Billing_Address', 'Billing_StreetNo', 'Billing_StreetName', 'Billing_StreetType',
                   'Billing_Suburb', 'Billing_State', 'Billing_Postcode', 'Billing_Customisable_Value', 'Billing_Address2']

billing_df = Addresses_Complete[billing_columns]

# Select the columns for delivery information and 'Customer_Code' and 'ID'
delivery_columns = ['Customer_Code', 'ID', 'Delivery_Address', 'Delivery_StreetNo', 'Delivery_StreetName', 'Delivery_StreetType',
                    'Delivery_Suburb', 'Delivery_State', 'Delivery_Postcode', 'Delivery_Customisable_Value', 'Delivery_Address2']

delivery_df = Addresses_Complete[delivery_columns]

In [82]:
billing_df = billing_df.dropna(subset=billing_df.columns[2:], how='all')

billing_df

,Customer_Code,ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,Billing_Customisable_Value,Billing_Address2
0,PBPHYIPLUMBING,11405,PO Box 5485 \nWOLLONGONG NSW 2520,None,PO Box 5485,None,WOLLONGONG,NSW,2520,None,None
1,9WAHBNWPXZENGINEERING,11289,PO Box 1161 \nMOUNTAIN GATE VIC 3165,None,PO Box 1161,None,MOUNTAIN GATE,VIC,3165,None,None
2,9EMKTVVGFJZXYUNWBUILDING,10999,T/As Odyssey Constructions & Fitout\nPO Box 71...,None,PO Box 7179,None,WETHERILL PARK BC,NSW,2164,T/As Odyssey Constructions & Fitout,None
3,9BGRBF,10677,PO BOX 13268\nGeorge Street\nQUEENSLAND QLD 4003,None,PO BOX 13268,None,QUEENSLAND,QLD,4003,None,George Street
4,TAXPTQPROJECTS,9625,Engineering Pty Ltd T/As CPB Ghella UGL JV\nAt...,None,PO Box 7261,None,ALEXANDRIA,NSW,2016,None,Engineering Pty Ltd T/As CPB Ghella UGL JV\nAt...
...,...,...,...,...,...,...,...,...,...,...,...
66,BRVPHIKKHWVHPROJECTS,887,Tower 2/ Level 21 101 Grafton St \nBONDI J...,Tower 2/ Level 21 101,Grafton St,None,BONDI JUNCTION,NSW,2022,None,None
67,DXFCKJUZMKIENGINEERING,777,"Suite 1, Level 1, 290 Botany Road\nALEXANDRIA ...","Suite 1, Level 1, 290",Botany Road,None,ALEXANDRIA,NSW,2015,None,None
68,ODCUJVZZGROUP,688,(Receivers & Managers Appointed) (in Liquidati...,"Unit 32, 11-21",Underwood Road,None,HOMEBUSH,NSW,2140,None,(Receivers & Managers Appointed) (in Liquidati...
69,NAGBHADCFDENGINEERING,596,C/-Broadcast Australia Transmitting Station\nF...,2-148,Sydenham Road,None,DELAHEY,VIC,3037,None,C/-Broadcast Australia Transmitting Station\nF...


## Inserting the addresses and retrieving the IDs

### Billing Addresses

This part checks if the suburb, postcode and state inserted by the user exists using the created functions explained before.

In [83]:
# Testing to see if the suburb and postcode macthes

billing_insert_statments = ''

check_file = ''

Billing_IDs_To_Skip = []

error_reason = ''

for index, row in billing_df.iterrows():
    
    suburb = row['Billing_Suburb']
    original_suburb = row['Billing_Suburb']
    postcode = row['Billing_Postcode']
    original_postcode = row['Billing_Postcode']
    state = row['Billing_State']
    customer_id = row['ID']
    
    
    if suburb == 'HRMC':
        suburb = 'HUNTER REGION MC'
    elif suburb == 'GCMC' or suburb == 'GCMC BUNDALL':
        suburb = 'GOLD COAST MC'
    elif suburb == 'QVB':
        suburb = 'QUEEN VICTORIA BUILDING'
    
    filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['Postcode'] == postcode)]
    
    
    if filtered_suburbs.empty:
        
        filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['StateName'] == state)]
        
        if len(filtered_suburbs)>1:
            best_postcode_diff = float('inf')
            best_match = None
            
            for _, current_row in filtered_suburbs.iterrows():
                current_postcode = current_row['Postcode']
                postcode_diff = abs(int(current_postcode) - int(original_postcode))

                if postcode_diff < best_postcode_diff:
                    best_postcode_diff = postcode_diff
                    suburb = current_row['SuburbName']
                    postcode = current_row['Postcode']
            
            filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['Postcode'] == postcode)]
            check_file += f'\nCheck 1\n'
        
        elif filtered_suburbs.empty:
            matching_suburb = find_matching_suburb(suburb, postcode, suburbs)
        
            if matching_suburb:
                suburb = matching_suburb
                filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb.upper()) & (suburbs['Postcode'] == postcode)]
                check_file += f'\nCheck 2\n'

            else:
                suburb = original_suburb
                matching_split = find_matching_suburb_split(suburb, postcode, suburbs)

                if matching_split:
                    suburb = matching_split
                    filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb.upper()) & (suburbs['Postcode'] == postcode)]
                    check_file += f'\nCheck 3\n'

                else:
                    suburb = original_suburb
                    matching_split_state = find_matching_suburb_split_state(suburb, state, postcode, suburbs)

                    if matching_split_state[0]:
                        suburb = matching_split_state[0]
                        postcode = matching_split_state[1]

                        filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper()== suburb.upper()) & (suburbs['Postcode'] == postcode)]
                        check_file += f'\nCheck 4\n'
            
            
            
    if filtered_suburbs.empty:
        check_file += f'\n {index}---{customer_id} --- {original_suburb} --- !!!!!!!!!!!!!!!! EMPTY !!!!!!!!!!!!!!!!! \n'
        Billing_IDs_To_Skip.append(customer_id)
        error_reason += 'Postcode and Suburb does not match.\n'
        
        
    else:
        suburb_id = filtered_suburbs['Id'].values[0]
        state_id = filtered_suburbs['StateID'].values[0]
        postcode = filtered_suburbs['Postcode'].values[0]
        suburb = filtered_suburbs['SuburbName'].values[0]
        
        # Getting the new postcode and suburb to update the billing address
        
        parts = row['Billing_Address'].split('\n') # spliting the string by _ that represents a line break
    
        billing_address = '\n'.join(parts[:-1])

        billing_address += f'\n{suburb} {state} {postcode}'
        
        
        street_number = row['Billing_StreetNo']
        
        if street_number is not None:
            len_stnumber = len(street_number)
            street_number = street_number.replace("'", "''") 
            street_number = street_number.replace("\n", "'+char(13)+char(10)+'")
            street_number = f"'{street_number}'"
        else:
            street_number = 'NULL'
            len_stnumber = len(street_number)

        
        street_name = row['Billing_StreetName']
        if street_name is not None:
            len_stname = len(street_name)
            street_name = street_name.replace("'", "''")
            street_name = street_name.replace("\n", "'+char(13)+char(10)+'")
            street_name = f"'{street_name}'"
        else:
            street_name = 'NULL'
            len_stname = len(street_name)
        
        address = billing_address
        if address is not None:
            len_add = len(address)
            address = address.replace("'", "''")
            address = address.replace("\n", "'+char(13)+char(10)+'")
        else:
            address = 'NULL'
            len_add = len(address)

        address1 = row['Billing_Customisable_Value']
        if address1 is not None:
            len_add1 = len(address1)
            address1 = address1.replace("'", "''")
            address1 = address1.replace("\n", "'+char(13)+char(10)+'")
            address1 = f"'{address1}'"
        else:
            address1 = 'NULL'
            len_add1 = len(address1)
            

        address2 = row['Billing_Address2']
        if address2 is not None:
            len_add2 = len(address2)
            address2 = address2.replace("'", "''")
            address2 = address2.replace("\n", "'+char(13)+char(10)+'")
            address2 = f"'{address2}'"
        else:
            address2 = 'NULL'
            len_add2 = len(address2)

        
        
        if postcode != original_postcode or suburb != original_suburb:
            check_file += f'\n!!!DIFFERENT!!!\nOriginal Suburb: {original_suburb} --- Original Postcode: {original_postcode}\nIndex: {index} \nMatching Sub: {suburb}\nPostcode: {postcode}\nID: {customer_id} \n'
        
        else:
            check_file += f'\nOriginal Suburb: {original_suburb} --- Original Postcode: {original_postcode}\nIndex: {index} \nMatching Sub: {suburb}\nPostcode: {postcode}\nID: {customer_id} \n'
    
        insert = f"""
        INSERT INTO [sample_database].[dbo].[ADR_Address]
          ([Address]
          ,[Address1]
          ,[StreetNumber]
          ,[StreetName]
          ,[Address2]
          ,[SuburbID]
          ,[StateID]
          ,[CountryID]
          ,[Postcode])
         VALUES
               ('{address}' ---- {f'Greater {customer_id}' if len_add > 1000 else 'Smaller'}
               ,{address1} ---- {f'Greater {customer_id}' if len_add1 > 250 else 'Smaller'}
               ,{street_number} ---- {f'Greater {customer_id}' if len_stnumber > 20 else 'Smaller'}
               ,{street_name} ---- {f'Greater {customer_id}' if len_stname > 100 else 'Smaller'}
               ,{address2} ---- {f'Greater {customer_id}' if len_add2 > 70 else 'Smaller'}
               ,{suburb_id}
               ,{state_id}
               ,36
               ,{postcode})
        """
        
        billing_insert_statments += insert
        
        
        if len_add > 1000:
            Billing_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the Billing_Address is longer than 1000 characters.\n'
        
        if len_add1 > 250:
            Billing_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the Billing_Customisable_Value is longer than 250 characters.\n'
        
        if len_stnumber > 20:
            Billing_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the StreetNumber is longer than 20 characters.\n'
        
        if len_stname > 100:
            Billing_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the StreetName is longer than 100 characters.\n'
            
        if len_add2 > 70:
            Billing_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the Address2 is longer than 70 characters.\n'
            
            
        if any([len_add > 1000, len_add1 > 250, len_stnumber > 20, len_stname > 100, len_stname > 100, len_add2 > 70]):
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_id,
                                                    'Billing_Address': row['Billing_Address'].replace("_", "\n"),
                                                    'Billing Error Type': error_reason
                        }])
            
            Billing_Error_Report = pd.concat([Billing_Error_Report, new_line],ignore_index=True)
                
        error_reason = ''

In [85]:
# Get the current date and time as a string
date_time = datetime.now().strftime("%Y%m%d_%H%M%S")

This part saves the values as they were before the modifications just in case we need to come back to them.

In [86]:
folder_path = f'\\\choose\\your\\path{date_time}'

#file_path = os.path.join(folder_path, f'Values_Before.xlsx')

#values_before_df.to_excel(file_path, index=False)

After the checks, the code inserts into the addresses table the fixed customer addresses and gets the generated ID to use in the next step.

In [87]:
# Establish the connection with Windows authentication

#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database
#cursor = conn.cursor()

billing_insert_statments = ''

check_file = ''


for index, row in billing_df.iterrows():
    
    
    suburb = row['Billing_Suburb']
    original_suburb = row['Billing_Suburb']
    postcode = row['Billing_Postcode']
    original_postcode = row['Billing_Postcode']
    state = row['Billing_State']
    customer_id = row['ID']
    
    if customer_id in Billing_IDs_To_Skip:
        continue
    
    if suburb == 'HRMC':
        suburb = 'HUNTER REGION MC'
    elif suburb == 'GCMC' or suburb == 'GCMC BUNDALL':
        suburb = 'GOLD COAST MC'
    elif suburb == 'QVB':
        suburb = 'QUEEN VICTORIA BUILDING'
    
    filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['Postcode'] == postcode)]
    
    
    if filtered_suburbs.empty:
        
        filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['StateName'] == state)]
        
        if len(filtered_suburbs)>1:
            best_postcode_diff = float('inf')
            best_match = None
            
            for _, current_row in filtered_suburbs.iterrows():
                current_postcode = current_row['Postcode']
                postcode_diff = abs(int(current_postcode) - int(original_postcode))

                if postcode_diff < best_postcode_diff:
                    best_postcode_diff = postcode_diff
                    suburb = current_row['SuburbName']
                    postcode = current_row['Postcode']
            
            filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['Postcode'] == postcode)]
        
        elif filtered_suburbs.empty:
            matching_suburb = find_matching_suburb(suburb, postcode, suburbs)
        
            if matching_suburb:
                suburb = matching_suburb
                filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb.upper()) & (suburbs['Postcode'] == postcode)]

            else:
                suburb = original_suburb
                matching_split = find_matching_suburb_split(suburb, postcode, suburbs)

                if matching_split:
                    suburb = matching_split
                    filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb.upper()) & (suburbs['Postcode'] == postcode)]

                else:
                    suburb = original_suburb
                    matching_split_state = find_matching_suburb_split_state(suburb, state, postcode, suburbs)

                    if matching_split_state[0]:
                        suburb = matching_split_state[0]
                        postcode = matching_split_state[1]

                        filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper()== suburb.upper()) & (suburbs['Postcode'] == postcode)]
            
            
    if filtered_suburbs.empty:
        continue
        
        
    else:
        suburb_id = filtered_suburbs['Id'].values[0]
        state_id = filtered_suburbs['StateID'].values[0]
        postcode = filtered_suburbs['Postcode'].values[0]
        suburb = filtered_suburbs['SuburbName'].values[0]
        
        # Getting the new postcode and suburb to update the billing address
        
        parts = row['Billing_Address'].split('\n') # spliting the string by _ that represents a line break
    
        billing_address = '\n'.join(parts[:-1])

        billing_address += f'\n{suburb} {state} {postcode}'
        
        
        street_number = row['Billing_StreetNo']
        
        if street_number is not None:
            street_number = street_number.replace("'", "''") 
            street_number = street_number.replace("\n", "'+char(13)+char(10)+'")
            street_number = f"'{street_number}'"
        else:
            street_number = 'NULL'
            len_stnumber = len(street_number)

        
        street_name = row['Billing_StreetName']
        if street_name is not None:
            street_name = street_name.replace("'", "''") 
            street_name = street_name.replace("\n", "'+char(13)+char(10)+'")
            street_name = f"'{street_name}'"
        else:
            street_name = 'NULL'
            len_stname = len(street_name)
        
        address = billing_address
        if address is not None:
            address = address.replace("'", "''")
            address = address.replace("\n", "'+char(13)+char(10)+'")
        else:
            address = 'NULL'

        address1 = row['Billing_Customisable_Value']
        if address1 is not None:
            address1 = address1.replace("'", "''")
            address1 = address1.replace("\n", "'+char(13)+char(10)+'")
            address1 = f"'{address1}'"
        else:
            address1 = 'NULL'
            len_add1 = len(address1)
            

        address2 = row['Billing_Address2']
        if address2 is not None:
            address2 = address2.replace("'", "''")
            address2 = address2.replace("\n", "'+char(13)+char(10)+'")
            address2 = f"'{address2}'"
        else:
            address2 = 'NULL'
            len_add2 = len(address2)

        
       
        insert = f"""
        INSERT INTO [sample_database].[dbo].[ADR_Address]
          ([Address]
          ,[Address1]
          ,[StreetNumber]
          ,[StreetName]
          ,[Address2]
          ,[SuburbID]
          ,[StateID]
          ,[CountryID]
          ,[Postcode])
         VALUES
               ('{address}'
               ,{address1} 
               ,{street_number} 
               ,{street_name}
               ,{address2}
               ,{suburb_id}
               ,{state_id}
               ,36
               ,{postcode})
        """
        
        billing_insert_statments += insert
        
        # Execute each SQL query in the list
        #cursor.execute(insert)

        #conn.commit()
        
        
        select_id = """
SELECT MAX([ID])
FROM [sample_database].[dbo].[ADR_Address]
        """
        # Execute the select_id query
        #cursor.execute(select_id)
        
        # Fetch the result and extract the ID
        #new_id = cursor.fetchone()[0]
        
        #Addresses_Complete.loc[Addresses_Complete['ID'] == customer_id, 'Billing_ADR_Address_ID'] = new_id
        Addresses_Complete.loc[Addresses_Complete['ID'] == customer_id, 'Billing_Suburb'] = suburb
        Addresses_Complete.loc[Addresses_Complete['ID'] == customer_id, 'Billing_Postcode'] = postcode
        Addresses_Complete.loc[Addresses_Complete['ID'] == customer_id, 'Billing_Address'] = billing_address
       
    
# print(new_id)

# Close the cursor and connection
#cursor.close()
#conn.close()

### Delivery Addresses

In [88]:
delivery_df = delivery_df.dropna(subset=delivery_df.columns[2:], how='all')

delivery_df

,Customer_Code,ID,Delivery_Address,Delivery_StreetNo,Delivery_StreetName,Delivery_StreetType,Delivery_Suburb,Delivery_State,Delivery_Postcode,Delivery_Customisable_Value,Delivery_Address2
0,PBPHYIPLUMBING,11405,"Suite 1, Level 6, 288 Forest Road \nHURSTVILL...","Suite 1, Level 6, 288",Forest Road,None,HURSTVILLE,NSW,2220,None,None
1,9WAHBNWPXZENGINEERING,11289,"Suite 2, Level 11/555 Lonsdale Street \nMELBOU...","Suite 2, Level 11/555",Lonsdale Street,None,MELBOURNE,VIC,3000,None,None
2,9EMKTVVGFJZXYUNWBUILDING,10999,T/As Odyssey Constructions & Fitout\nSuite 1E ...,Suite 1E Level 1 1345,The Horsley Drive,None,WETHERILL PARK,NSW,2164,T/As Odyssey Constructions & Fitout,None
4,TAXPTQPROJECTS,9625,Engineering Pty Ltd T/As CPB Ghella UGL JV\nLe...,Level 7 197-201,Coward Street,None,MASCOT,NSW,2020,None,Engineering Pty Ltd T/As CPB Ghella UGL JV
5,ZWWGFVULCXOHSERVICES,9384,"Endeavour House, 11-15 Fourth Avenue\nMAWSON L...","Endeavour House, 11-15",Fourth Avenue,None,MAWSON LAKES,SA,5095,None,None
...,...,...,...,...,...,...,...,...,...,...,...
126,A1MYY3VUE1,1024,Cnr Five Islands & Flinders Street\nPORT KEMBL...,,Cnr Five Islands & Flinders Street,None,PORT KEMBLA,NSW,2505,None,None
127,NSNVUW4LWO,781,Cnr of Kerr & Aero Roads\nINGLEBURN NSW 2565,,Cnr of Kerr & Aero Roads,None,INGLEBURN,NSW,2565,None,None
128,BKBHQFGT54,554,2388 Silverdale Road\nSILVERDALE NSW 2752,2388,Silverdale Road,None,SILVERDALE,NSW,2752,None,None
129,FH9YG6L9UT,327,Cnr Stockton & Dredge Avenues\nMOOREBANK NSW 2170,,Cnr Stockton & Dredge Avenues,None,MOOREBANK,NSW,2170,None,None


In [89]:
delivery_insert_statments = ''

delivery_check_file = ''

Delivery_IDs_To_Skip = []

error_reason = ''

for index, row in delivery_df.iterrows():
    
    suburb = row['Delivery_Suburb']
    original_suburb = row['Delivery_Suburb']
    postcode = row['Delivery_Postcode']
    original_postcode = row['Delivery_Postcode']
    state = row['Delivery_State']
    customer_id = row['ID']
    
    if suburb == 'HRMC':
        suburb = 'HUNTER REGION MC'
    elif suburb == 'GCMC' or suburb == 'GCMC BUNDALL':
        suburb = 'GOLD COAST MC'
    elif suburb == 'QVB':
        suburb = 'QUEEN VICTORIA BUILDING'
        
    
    filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['Postcode'] == postcode)]
    
    
    if filtered_suburbs.empty:
        
        filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['StateName'] == state)]
        
        if len(filtered_suburbs)>1:
            best_postcode_diff = float('inf')
            best_match = None
            
            for _, current_row in filtered_suburbs.iterrows():
                current_postcode = current_row['Postcode']
                postcode_diff = abs(int(current_postcode) - int(original_postcode))

                if postcode_diff < best_postcode_diff:
                    best_postcode_diff = postcode_diff
                    suburb = current_row['SuburbName']
                    postcode = current_row['Postcode']
            
            filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['Postcode'] == postcode)]
            delivery_check_file += f'\nCheck 1\n'
        
        elif filtered_suburbs.empty:
            matching_suburb = find_matching_suburb(suburb, postcode, suburbs)
        
            if matching_suburb:
                suburb = matching_suburb
                filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb.upper()) & (suburbs['Postcode'] == postcode)]
                delivery_check_file += f'\nCheck 2\n'

            else:
                suburb = original_suburb
                matching_split = find_matching_suburb_split(suburb, postcode, suburbs)

                if matching_split:
                    suburb = matching_split
                    filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb.upper()) & (suburbs['Postcode'] == postcode)]
                    delivery_check_file += f'\nCheck 3\n'

                else:
                    suburb = original_suburb
                    matching_split_state = find_matching_suburb_split_state(suburb, state, postcode, suburbs)

                    if matching_split_state[0]:
                        suburb = matching_split_state[0]
                        postcode = matching_split_state[1]

                        filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper()== suburb.upper()) & (suburbs['Postcode'] == postcode)]
                        delivery_check_file += f'\nCheck 4\n'
            
            
            
    if filtered_suburbs.empty:
        delivery_check_file += f'\n {index}---{customer_id} --- {original_suburb} --- !!!!!!!!!!!!!!!! EMPTY !!!!!!!!!!!!!!!!! \n'
        Delivery_IDs_To_Skip.append(customer_id)
        error_reason += 'Postcode and Suburb does not match.\n'
        
        
    else:
        suburb_id = filtered_suburbs['Id'].values[0]
        state_id = filtered_suburbs['StateID'].values[0]
        postcode = filtered_suburbs['Postcode'].values[0]
        suburb = filtered_suburbs['SuburbName'].values[0]
        
        # Getting the new postcode and suburb to update the billing address
        
        parts = row['Delivery_Address'].split('\n') # spliting the string by _ that represents a line break
    
        delivery_address = '\n'.join(parts[:-1])

        delivery_address += f'\n{suburb} {state} {postcode}'
        
        
        street_number = row['Delivery_StreetNo']
        
        if street_number is not None:
            len_stnumber = len(street_number)
            street_number = street_number.replace("'", "''") 
            street_number = street_number.replace("\n", "'+char(13)+char(10)+'")
            street_number = f"'{street_number}'"
        else:
            street_number = 'NULL'
            len_stnumber = len(street_number)
        
        street_name = row['Delivery_StreetName']
        if street_name is not None:
            len_stname = len(street_name)
            street_name = street_name.replace("'", "''") 
            street_name = street_name.replace("\n", "'+char(13)+char(10)+'")
            street_name = f"'{street_name}'"
        else:
            street_name = 'NULL'
            len_stname = len(street_name)
        
        address = delivery_address
        if address is not None:
            len_add = len(address)
            address = address.replace("'", "''")
            address = address.replace("\n", "'+char(13)+char(10)+'")
        else:
            address = 'NULL'
            len_add = len(address)

        address1 = row['Delivery_Customisable_Value']
        if address1 is not None:
            len_add1 = len(address1)
            address1 = address1.replace("'", "''")
            address1 = address1.replace("\n", "'+char(13)+char(10)+'")
            address1 = f"'{address1}'"
        else:
            address1 = 'NULL'
            len_add1 = len(address1)

        address2 = row['Delivery_Address2']
        if address2 is not None:
            len_add2 = len(address2)
            address2 = address2.replace("'", "''")
            address2 = address2.replace("\n", "'+char(13)+char(10)+'")
            address2 = f"'{address2}'"
        else:
            address2 = 'NULL'
            len_add2 = len(address2)

        
        
        if postcode != original_postcode or suburb != original_suburb:
            delivery_check_file += f'\n!!!DIFFERENT!!!\nOriginal Suburb: {original_suburb} --- Original Postcode: {original_postcode}\nIndex: {index} \nMatching Sub: {suburb}\nPostcode: {postcode}\nID: {customer_id} \n'
        
        else:
            delivery_check_file += f'\nOriginal Suburb: {original_suburb} --- Original Postcode: {original_postcode}\nIndex: {index} \nMatching Sub: {suburb}\nPostcode: {postcode}\nID: {customer_id} \n'
    
        insert = f"""
        INSERT INTO [sample_database].[dbo].[ADR_Address]
          ([Address]
          ,[Address1]
          ,[StreetNumber]
          ,[StreetName]
          ,[Address2]
          ,[SuburbID]
          ,[StateID]
          ,[CountryID]
          ,[Postcode])
         VALUES
               ('{address}' ---- {f'Greater {customer_id}' if len(address) > 1000 else 'Smaller'}
               ,{address1} ---- {f'Greater {customer_id}' if len_add1 > 250 else 'Smaller'}
               ,{street_number} ---- {f'Greater {customer_id}' if len_stnumber > 20 else 'Smaller'}
               ,{street_name} ---- {f'Greater {customer_id}' if len_stname > 100 else 'Smaller'}
               ,{address2} ---- {f'Greater {customer_id}' if len_add2 > 70 else 'Smaller'}
               ,{suburb_id}
               ,{state_id}
               ,36
               ,{postcode})
        """
        
        delivery_insert_statments += insert
             
        
        if len_add > 1000:
            Delivery_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the Billing_Address is longer than 1000 characters.\n'
        
        if len_add1 > 250:
            Delivery_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the Billing_Customisable_Value is longer than 250 characters.\n'
        
        if len_stnumber > 20:
            Delivery_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the StreetNumber is longer than 20 characters.\n'
        
        if len_stname > 100:
            Delivery_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the StreetName is longer than 100 characters.\n'
            
        if len_add2 > 70:
            Delivery_IDs_To_Skip.append(customer_id)
            error_reason += 'Lenght of the Address2 is longer than 70 characters.\n'
            
            
        if any([len_add > 1000, len_add1 > 250, len_stnumber > 20, len_stname > 100, len_stname > 100, len_add2 > 70]):
            new_line = pd.DataFrame.from_records([{'Customer_Code': row['Customer_Code'],
                                                    'ID': customer_id,
                                                    'Delivery_Address': row['Delivery_Address'].replace("_", "\n"),
                                                    'Delivery Error Type': error_reason
                        }])
            
            Delivery_Error_Report = pd.concat([Delivery_Error_Report, new_line],ignore_index=True)
                
        error_reason = ''

This part was commented because it only works when a file is saved.

It saves the Error Report in an Excel file so it can be checked after what happened in each customer.

In [90]:
folder_path = f'\\\choose\\your\\path\\{date_time}'

'''
file_path = os.path.join(folder_path, f'Error_Report.xlsx')

Error_Report = pd.merge(Billing_Error_Report, Delivery_Error_Report, on=['Customer_Code', 'ID'], how='outer')

error_report_columns = ['Customer_Code', 'ID'] + [col for col in Error_Report.columns if col not in ['Customer_Code', 'ID']]
Error_Report = Error_Report[error_report_columns]

Error_Report.to_excel(file_path, index=False)

# Create ExcelWriter and write DataFrame to Excel
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    Error_Report.to_excel(writer, index=False, sheet_name='Sheet1')

    # Access the xlsxwriter workbook and worksheet objects from the ExcelWriter object.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Add a text wrap format.
    text_wrap_format = workbook.add_format({'text_wrap': True, 'align': 'center'})
    
    # Set the width of column 'A' to 26
    worksheet.set_column('A:A', 26)
    
    # Set the width of column 'B' to 10
    worksheet.set_column('B:B', 10)
    
    # Set the width of columns 'C' to 'F' to 55
    worksheet.set_column('C:F', 55, text_wrap_format)

'''

"\nfile_path = os.path.join(folder_path, f'Error_Report.xlsx')\n\nError_Report = pd.merge(Billing_Error_Report, Delivery_Error_Report, on=['Customer_Code', 'ID'], how='outer')\n\nerror_report_columns = ['Customer_Code', 'ID'] + [col for col in Error_Report.columns if col not in ['Customer_Code', 'ID']]\nError_Report = Error_Report[error_report_columns]\n\nError_Report.to_excel(file_path, index=False)\n\n# Create ExcelWriter and write DataFrame to Excel\nwith pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:\n    Error_Report.to_excel(writer, index=False, sheet_name='Sheet1')\n\n    # Access the xlsxwriter workbook and worksheet objects from the ExcelWriter object.\n    workbook = writer.book\n    worksheet = writer.sheets['Sheet1']\n\n    # Add a text wrap format.\n    text_wrap_format = workbook.add_format({'text_wrap': True, 'align': 'center'})\n    \n    # Set the width of column 'A' to 26\n    worksheet.set_column('A:A', 26)\n    \n    # Set the width of column 'B' to 10\n 

The next part will open the Error Report dataset and check if there's any line.

Case there are error lines, the code sends a request to a link an activate a PowerAutomate flow that get the error report file and sends via email to the responsible users.

In [91]:
# Check if the Error_Report DataFrame has any lines

'''
if not Error_Report.empty:
    # The DataFrame has lines, proceed with further actions
    folder_path = f'\\\choose\\your\\path\\Error Report'
    
    #file_path = os.path.join(folder_path, f'Error_Report.xlsx')

    error_report_columns = ['Customer_Code', 'ID'] + [col for col in Error_Report.columns if col not in ['Customer_Code', 'ID']]
    Error_Report = Error_Report[error_report_columns]

    Error_Report.to_excel(file_path, index=False)

    # Create ExcelWriter and write DataFrame to Excel
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        Error_Report.to_excel(writer, index=False, sheet_name='Sheet1')

        # Access the xlsxwriter workbook and worksheet objects from the ExcelWriter object.
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']

        # Add a text wrap format.
        text_wrap_format = workbook.add_format({'text_wrap': True, 'align': 'center'})
        
        # Set the width of column 'A' to 26
        worksheet.set_column('A:A', 26)
        
        # Set the width of column 'B' to 10
        worksheet.set_column('B:B', 10)
        
        # Set the width of columns 'C' to 'F' to 55
        worksheet.set_column('C:F', 55, text_wrap_format)
        
    time.sleep(5)
        
    url = 'write here the power automate flow link to activate the flow'

    # Send POST request to Power Automate flow
    requests.post(url)
    
'''

"\nif not Error_Report.empty:\n    # The DataFrame has lines, proceed with further actions\n    folder_path = f'\\\\choose\\your\\path\\Error Report'\n    \n    #file_path = os.path.join(folder_path, f'Error_Report.xlsx')\n\n    error_report_columns = ['Customer_Code', 'ID'] + [col for col in Error_Report.columns if col not in ['Customer_Code', 'ID']]\n    Error_Report = Error_Report[error_report_columns]\n\n    Error_Report.to_excel(file_path, index=False)\n\n    # Create ExcelWriter and write DataFrame to Excel\n    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:\n        Error_Report.to_excel(writer, index=False, sheet_name='Sheet1')\n\n        # Access the xlsxwriter workbook and worksheet objects from the ExcelWriter object.\n        workbook = writer.book\n        worksheet = writer.sheets['Sheet1']\n\n        # Add a text wrap format.\n        text_wrap_format = workbook.add_format({'text_wrap': True, 'align': 'center'})\n        \n        # Set the width of colum

Now it inserts the delivery addresses values into the addresses table and gets the generated id to use in the next step. 

In [92]:
# Establish the connection with Windows authentication

#conn = pyodbc.connect(
#    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
#)

# Create a cursor object to interact with the database
#cursor = conn.cursor()

delivery_insert_statments = ''

delivery_check_file = ''


for index, row in delivery_df.iterrows():
    
    
    suburb = row['Delivery_Suburb']
    original_suburb = row['Delivery_Suburb']
    postcode = row['Delivery_Postcode']
    original_postcode = row['Delivery_Postcode']
    state = row['Delivery_State']
    customer_id = row['ID']
    
    if customer_id in Delivery_IDs_To_Skip:
        continue
    
    if suburb == 'HRMC':
        suburb = 'HUNTER REGION MC'
    elif suburb == 'GCMC' or suburb == 'GCMC BUNDALL':
        suburb = 'GOLD COAST MC'
    elif suburb == 'QVB':
        suburb = 'QUEEN VICTORIA BUILDING'
    
    filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['Postcode'] == postcode)]
    
    
    if filtered_suburbs.empty:
        
        filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['StateName'] == state)]
        
        if len(filtered_suburbs)>1:
            best_postcode_diff = float('inf')
            best_match = None
            
            for _, current_row in filtered_suburbs.iterrows():
                current_postcode = current_row['Postcode']
                postcode_diff = abs(int(current_postcode) - int(original_postcode))

                if postcode_diff < best_postcode_diff:
                    best_postcode_diff = postcode_diff
                    suburb = current_row['SuburbName']
                    postcode = current_row['Postcode']
            
            filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb) & (suburbs['Postcode'] == postcode)]
        
        elif filtered_suburbs.empty:
            matching_suburb = find_matching_suburb(suburb, postcode, suburbs)
        
            if matching_suburb:
                suburb = matching_suburb
                filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb.upper()) & (suburbs['Postcode'] == postcode)]

            else:
                suburb = original_suburb
                matching_split = find_matching_suburb_split(suburb, postcode, suburbs)

                if matching_split:
                    suburb = matching_split
                    filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper() == suburb.upper()) & (suburbs['Postcode'] == postcode)]

                else:
                    suburb = original_suburb
                    matching_split_state = find_matching_suburb_split_state(suburb, state, postcode, suburbs)

                    if matching_split_state[0]:
                        suburb = matching_split_state[0]
                        postcode = matching_split_state[1]

                        filtered_suburbs = suburbs[(suburbs['SuburbName'].str.upper()== suburb.upper()) & (suburbs['Postcode'] == postcode)]
            
            
            
    if filtered_suburbs.empty:
        continue
        
    else:
        suburb_id = filtered_suburbs['Id'].values[0]
        state_id = filtered_suburbs['StateID'].values[0]
        postcode = filtered_suburbs['Postcode'].values[0]
        suburb = filtered_suburbs['SuburbName'].values[0]
        
        # Getting the new postcode and suburb to update the billing address
        
        parts = row['Delivery_Address'].split('\n') # spliting the string by _ that represents a line break
    
        delivery_address = '\n'.join(parts[:-1])

        delivery_address += f'\n{suburb} {state} {postcode}'
        
        
        street_number = row['Delivery_StreetNo']
        
        if street_number is not None:
            len_stnumber = len(street_number)
            street_number = street_number.replace("'", "''") 
            street_number = street_number.replace("\n", "'+char(13)+char(10)+'")
            street_number = f"'{street_number}'"
        else:
            street_number = 'NULL'
            len_stnumber = len(street_number)
        
        street_name = row['Delivery_StreetName']
        if street_name is not None:
            len_stname = len(street_name)
            street_name = street_name.replace("'", "''") 
            street_name = street_name.replace("\n", "'+char(13)+char(10)+'")
            street_name = f"'{street_name}'"
        else:
            street_name = 'NULL'
            len_stname = len(street_name)
        
        address = delivery_address
        if address is not None:
            address = address.replace("'", "''")
            address = address.replace("\n", "'+char(13)+char(10)+'")
        else:
            address = 'NULL'

        address1 = row['Delivery_Customisable_Value']
        if address1 is not None:
            len_add1 = len(address1)
            address1 = address1.replace("'", "''")
            address1 = address1.replace("\n", "'+char(13)+char(10)+'")
            address1 = f"'{address1}'"
        else:
            address1 = 'NULL'
            len_add1 = len(address1)

        address2 = row['Delivery_Address2']
        if address2 is not None:
            len_add2 = len(address2)
            address2 = address2.replace("'", "''")
            address2 = address2.replace("\n", "'+char(13)+char(10)+'")
            address2 = f"'{address2}'"
        else:
            address2 = 'NULL'
            len_add2 = len(address2)

        
        
        
        insert = f"""
        INSERT INTO [sample_database].[dbo].[ADR_Address]
          ([Address]
          ,[Address1]
          ,[StreetNumber]
          ,[StreetName]
          ,[Address2]
          ,[SuburbID]
          ,[StateID]
          ,[CountryID]
          ,[Postcode])
         VALUES
               ('{address}'
               ,{address1}
               ,{street_number}
               ,{street_name}
               ,{address2}
               ,{suburb_id}
               ,{state_id}
               ,36
               ,{postcode})
        """
        
        delivery_insert_statments += insert
        
        # Execute each SQL query in the list
        
        #cursor.execute(insert)

        #conn.commit()
        
        
        select_id = """
SELECT MAX([ID])
FROM [sample_database].[dbo].[ADR_Address]
        """
        # Execute the select_id query
        #cursor.execute(select_id)
        
        # Fetch the result and extract the ID
        #new_id = cursor.fetchone()[0]
        
        #Addresses_Complete.loc[Addresses_Complete['ID'] == customer_id, 'Delivery_ADR_Address_ID'] = new_id
        Addresses_Complete.loc[Addresses_Complete['ID'] == customer_id, 'Delivery_Suburb'] = suburb
        Addresses_Complete.loc[Addresses_Complete['ID'] == customer_id, 'Delivery_Postcode'] = postcode
        Addresses_Complete.loc[Addresses_Complete['ID'] == customer_id, 'Delivery_Address'] = delivery_address
        
#print(new_id)

# Close the cursor and connection

#cursor.close()
#conn.close()

In [93]:
# Set billing columns to None if ID is in Billing_IDs_To_Skip
Addresses_Complete.loc[Addresses_Complete['ID'].isin(Billing_IDs_To_Skip), [
    'Billing_ADR_Address_ID', 'Billing_Address', 'Billing_StreetNo',
    'Billing_StreetName', 'Billing_StreetType', 'Billing_Suburb',
    'Billing_State', 'Billing_Postcode', 'Billing_Customisable_Value',
    'Billing_Address2'
]] = None

# Set delivery columns to None if ID is in Delivery_IDs_To_Skip
Addresses_Complete.loc[Addresses_Complete['ID'].isin(Delivery_IDs_To_Skip), [
    'Delivery_ADR_Address_ID', 'Delivery_Address', 'Delivery_StreetNo',
    'Delivery_StreetName', 'Delivery_StreetType', 'Delivery_Suburb',
    'Delivery_State', 'Delivery_Postcode', 'Delivery_Customisable_Value',
    'Delivery_Address2'
]] = None


Addresses_Complete = Addresses_Complete.dropna(subset = Addresses_Complete.columns[2:], how='all')

Addresses_Complete.reset_index(drop = True, inplace = True)

Next steo saves the complete addresses table just for consultation if needed.

In [94]:
folder_path = f'\\\choose\\your\\path\\{date_time}'

#file_path = os.path.join(folder_path, f'Addresses_Complete_Auto.xlsx')

#Addresses_Complete.to_excel(file_path, index=False)

## Updating the Master table

### Billing Fields

In [95]:
billing_columns2 = ['Customer_Code', 'ID', 'Billing_ADR_Address_ID', 'Billing_Address', 'Billing_StreetNo', 'Billing_StreetName', 'Billing_StreetType',
                   'Billing_Suburb', 'Billing_State', 'Billing_Postcode', 'Billing_Customisable_Value', 'Billing_Address2']

# Select the columns for delivery information and 'Customer_Code' and 'ID'
delivery_columns2 = ['Customer_Code', 'ID', 'Delivery_ADR_Address_ID', 'Delivery_Address', 'Delivery_StreetNo', 'Delivery_StreetName', 'Delivery_StreetType',
                    'Delivery_Suburb', 'Delivery_State', 'Delivery_Postcode', 'Delivery_Customisable_Value', 'Delivery_Address2']

In [96]:
Addresses_Complete_Billing = Addresses_Complete[billing_columns2]

Addresses_Complete_Billing = Addresses_Complete_Billing.dropna(subset = Addresses_Complete_Billing.columns[2:], how='all')

Addresses_Complete_Billing.reset_index(drop = True, inplace = True)

In [97]:
Addresses_Complete_Billing

,Customer_Code,ID,Billing_ADR_Address_ID,Billing_Address,Billing_StreetNo,Billing_StreetName,Billing_StreetType,Billing_Suburb,Billing_State,Billing_Postcode,Billing_Customisable_Value,Billing_Address2
0,PBPHYIPLUMBING,11405,None,PO Box 5485 \nWOLLONGONG NSW 2520,None,PO Box 5485,None,WOLLONGONG,NSW,2520,None,None
1,9WAHBNWPXZENGINEERING,11289,None,PO Box 1161 \nMOUNTAIN GATE VIC 3156,None,PO Box 1161,None,MOUNTAIN GATE,VIC,3156,None,None
2,9EMKTVVGFJZXYUNWBUILDING,10999,None,T/As Odyssey Constructions & Fitout\nPO Box 71...,None,PO Box 7179,None,WETHERILL PARK,NSW,2164,T/As Odyssey Constructions & Fitout,None
3,9BGRBF,10677,None,PO BOX 13268\nGeorge Street\nQueensland Univer...,None,PO BOX 13268,None,Queensland University of Tech,QLD,4000,None,George Street
4,ZWWGFVULCXOHSERVICES,9384,None,PO Box 86\nPOORAKA SA 5095,None,PO Box 86,None,POORAKA,SA,5095,None,None
5,FOSTLYRCPROJECTS,8954,None,PO Box 567\nCARINA HEIGHTS QLD 4152,None,PO Box 567,None,CARINA HEIGHTS,QLD,4152,None,None
6,UECRDNBBAYENGINEERING,6935,None,PO Box 3355\nDARRA QLD 4076,None,PO Box 3355,None,DARRA,QLD,4076,None,None
7,NYULZQAGEEKPY,6519,None,P.O Box 1410\nBEENLEIGH QLD 4207,None,P.O Box 1410,None,BEENLEIGH,QLD,4207,None,None
8,UNDHWRBUILDING,4795,None,PO Box 3346\nRICHMOND VIC 3121,None,PO Box 3346,None,RICHMOND,VIC,3121,None,None
9,9HKLCDEYVTKQH,4511,None,PO Box 3110\nLANSVALE NSW 2166,None,PO Box 3110,None,LANSVALE,NSW,2166,None,None


After all changes and fixes, the code updates the customer table in SQL to reflect the correct values.

In [98]:
'''
# Establish the connection with Windows authentication
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
)

# Create a cursor object to interact with the database
cursor = conn.cursor()

all_statments = ''


for index, row in Addresses_Complete_Billing.iterrows():
    
    customer_id = row['ID']

    if customer_id in (Billing_IDs_To_Skip + Billing_IDs_To_Exclude):
        continue
    
    update_string = ''
    count = 0
    
    for column in billing_columns2[2:]:
        
        value = Addresses_Complete_Billing.at[index, column]

        if value is not None:
            value = str(value)
            value = value.replace("'", "''")  # Escape single quotes
            value = value.replace("\n", "'+char(13)+char(10)+'")
    
            
            if count == 0:
                update_string += f"[{column}] = '{value}'\n"
                count += 1
            else:
                update_string += f",[{column}] = '{value}'\n"
                count += 1
    
    update_beginning = """UPDATE [sample_database].[dbo].[DBT_Master]
SET
 """
    
    update_end = f""",[Billing_Country] = 'Australia'
    WHERE [ID] = {customer_id};
    """
    
    
    sql_query = update_beginning + update_string + update_end
    
    all_statments += sql_query
    
    # Execute each SQL query in the list
    cursor.execute(sql_query)

    conn.commit()
    
# Close the cursor and connection
cursor.close()
conn.close()
'''

'\n# Establish the connection with Windows authentication\nconn = pyodbc.connect(\n    f\'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};\'\n)\n\n# Create a cursor object to interact with the database\ncursor = conn.cursor()\n\nall_statments = \'\'\n\n\nfor index, row in Addresses_Complete_Billing.iterrows():\n    \n    customer_id = row[\'ID\']\n\n    if customer_id in (Billing_IDs_To_Skip + Billing_IDs_To_Exclude):\n        continue\n    \n    update_string = \'\'\n    count = 0\n    \n    for column in billing_columns2[2:]:\n        \n        value = Addresses_Complete_Billing.at[index, column]\n\n        if value is not None:\n            value = str(value)\n            value = value.replace("\'", "\'\'")  # Escape single quotes\n            value = value.replace("\n", "\'+char(13)+char(10)+\'")\n    \n            \n            if count == 0:\n                update_string += f"[{column}] = \'{value}\'\n"\n                coun

### Delivery Fields

In [99]:
Addresses_Complete_Delivery = Addresses_Complete[delivery_columns2]

Addresses_Complete_Delivery = Addresses_Complete_Delivery.dropna(subset = Addresses_Complete_Delivery.columns[2:], how='all')

Addresses_Complete_Delivery.reset_index(drop = True, inplace = True)

In [100]:
'''
# Establish the connection with Windows authentication
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
)

# Create a cursor object to interact with the database
cursor = conn.cursor()


for index, row in Addresses_Complete_Delivery.iterrows():
    
    customer_id = row['ID']

    if customer_id in (Delivery_IDs_To_Skip + Delivery_IDs_To_Exclude):
        continue
    
    update_string = ''
    count = 0
    
    for column in delivery_columns2[2:]:
        
        value = Addresses_Complete_Delivery.at[index, column]

        if value is not None:
            value = str(value)
            value = value.replace("'", "''")  # Escape single quotes
            value = value.replace("\n", "'+char(13)+char(10)+'")
    
            
            if count == 0:
                update_string += f"[{column}] = '{value}'\n"
                count += 1
            else:
                update_string += f",[{column}] = '{value}'\n"
                count += 1
    
    update_beginning = """UPDATE [sample_database].[dbo].[DBT_Master]
SET
 """
    
    update_end = f""",[Delivery_Country] = 'Australia'
    WHERE [ID] = {customer_id};
    """
    
    
    sql_query = update_beginning + update_string + update_end
    
    all_statments += sql_query
    
    # Execute each SQL query in the list
    cursor.execute(sql_query)

    conn.commit()
    
# Close the cursor and connection
cursor.close()
conn.close()
'''

'\n# Establish the connection with Windows authentication\nconn = pyodbc.connect(\n    f\'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};\'\n)\n\n# Create a cursor object to interact with the database\ncursor = conn.cursor()\n\n\nfor index, row in Addresses_Complete_Delivery.iterrows():\n    \n    customer_id = row[\'ID\']\n\n    if customer_id in (Delivery_IDs_To_Skip + Delivery_IDs_To_Exclude):\n        continue\n    \n    update_string = \'\'\n    count = 0\n    \n    for column in delivery_columns2[2:]:\n        \n        value = Addresses_Complete_Delivery.at[index, column]\n\n        if value is not None:\n            value = str(value)\n            value = value.replace("\'", "\'\'")  # Escape single quotes\n            value = value.replace("\n", "\'+char(13)+char(10)+\'")\n    \n            \n            if count == 0:\n                update_string += f"[{column}] = \'{value}\'\n"\n                count += 1\n           

In [101]:
# Set the finish date and time to the current moment
finish_date = datetime.now()

# Calculate the time spent
time_spent = finish_date - start_date

# Format dates and times as strings in the desired format
start_date_str = start_date.strftime('%d/%m/%y %H:%M:%S')
finish_date_str = finish_date.strftime('%d/%m/%y %H:%M:%S')

Lastly, it inserts into the log table the information of this run.

In [102]:
'''
# Establish the connection with Windows authentication
conn = pyodbc.connect(
    f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};'
)

# Create a cursor object to interact with the database
cursor = conn.cursor()

start_date_SQL = start_date.strftime('%Y-%m-%d %H:%M:%S')
finish_date_SQL = finish_date.strftime('%Y-%m-%d %H:%M:%S')

if last_modified_date:
    last_modified = last_modified_date
    
else:
    last_modified = last_finish_date

no_rows = Addresses_Complete.shape[0]

insert_log = f"""

INSERT INTO [sample_database].[dbo].[TBL_DBT_Address_Updates_Log] (StartDate, FinishDate, TotalTime, No_Rows, Last_Modified_Date)
VALUES ('{start_date_SQL}', '{finish_date_SQL}', '{time_spent}', {no_rows}, '{last_modified}');

"""

cursor.execute(insert_log)

conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()
'''

'\n# Establish the connection with Windows authentication\nconn = pyodbc.connect(\n    f\'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};\'\n)\n\n# Create a cursor object to interact with the database\ncursor = conn.cursor()\n\nstart_date_SQL = start_date.strftime(\'%Y-%m-%d %H:%M:%S\')\nfinish_date_SQL = finish_date.strftime(\'%Y-%m-%d %H:%M:%S\')\n\nif last_modified_date:\n    last_modified = last_modified_date\n    \nelse:\n    last_modified = last_finish_date\n\nno_rows = Addresses_Complete.shape[0]\n\ninsert_log = f"""\n\nINSERT INTO [sample_database].[dbo].[TBL_DBT_Address_Updates_Log] (StartDate, FinishDate, TotalTime, No_Rows, Last_Modified_Date)\nVALUES (\'{start_date_SQL}\', \'{finish_date_SQL}\', \'{time_spent}\', {no_rows}, \'{last_modified}\');\n\n"""\n\ncursor.execute(insert_log)\n\nconn.commit()\n\n# Close the cursor and connection\ncursor.close()\nconn.close()\n'